# Setup

## Function to read iMotions sensor file

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd

from wbdlib import read_imotions, read_imotions_metadata

project_root = Path.cwd().parent
data_export_dir = project_root / "data" / "Export"

## Explanation of functions

The above functions are used to read in the sesor data files, one csv at a time, and extract single features per stimulus, and write these features to a simple results file.

The functions must be adjusted to:
- Discern between long form and short form
- Isolate key moments from timings file provided by client
- Extract time series
- Compute group-wide features such as inter-subject correlation

# Preparation
- Create naming dictionary for all stims
- Get total times of all stims
- Prepare key_moments

In [17]:
# Locate one sensor export per group for duration scanning


group_sensor_files = {}
for group_dir in sorted(data_export_dir.glob("Group *")):
    if not group_dir.is_dir():
        continue
    sensor_dirs = sorted(group_dir.glob("Analyses/*/Sensor Data"))
    csv_candidates = []
    for sensor_dir in sensor_dirs:
        csv_candidates.extend(sorted(sensor_dir.glob("*.csv")))
    group_sensor_files[group_dir.name] = csv_candidates[0] if csv_candidates else None

sensor_selection = pd.DataFrame([
    {
        "group": group,
        "sensor_file": path.name if path else None
    }
    for group, path in group_sensor_files.items()
]).sort_values("group").reset_index(drop=True)

sensor_selection

,group,sensor_file
0,Group A,001_116.csv
1,Group B,001_58.csv
2,Group C,001_114.csv
3,Group D,001_102.csv
4,Group E,001_108.csv
5,Group F,001_107.csv


In [18]:
# Collect per-group stimulus durations without aggregating across groups
duration_tables = []
issues = {}

for group, path in group_sensor_files.items():
    if path is None:
        issues[group] = "No sensor CSV found"
        continue
    try:
        df_group, _ = read_imotions(path)
    except Exception as exc:
        issues[group] = f"read_imotions failed: {exc}"
        continue

    required_cols = {"SourceStimuliName", "Timestamp"}
    if not required_cols.issubset(df_group.columns):
        issues[group] = "Missing SourceStimuliName or Timestamp"
        continue
    df_clean = df_group[["SourceStimuliName", "Timestamp"]].copy()
    df_clean = df_clean.dropna(subset=["SourceStimuliName"])
    df_clean["Timestamp"] = pd.to_numeric(df_clean["Timestamp"], errors="coerce")
    df_clean = df_clean.dropna(subset=["Timestamp"])
    if df_clean.empty:
        issues[group] = "No valid timestamp data"
        continue

    group_duration = (
        df_clean.groupby("SourceStimuliName")["Timestamp"]
        .apply(lambda s: s.max() - s.min())
        .reset_index(name="duration_ms")
    )

    if group_duration.empty:
        issues[group] = "No stimuli with duration"
        continue

    group_duration["duration_seconds"] = group_duration["duration_ms"] / 1000.0
    group_duration["duration_minutes"] = group_duration["duration_seconds"] / 60.0
    group_duration.insert(0, "group", group)
    group_duration.rename(columns={"SourceStimuliName": "stimulus_name"}, inplace=True)
    duration_tables.append(group_duration[["group", "stimulus_name", "duration_seconds", "duration_minutes"]])

if duration_tables:
    stimulus_summary = pd.concat(duration_tables, ignore_index=True)
    stimulus_summary.sort_values(["group", "stimulus_name"], inplace=True)
    stimulus_summary["duration_seconds"] = stimulus_summary["duration_seconds"].round(2)
    stimulus_summary["duration_minutes"] = stimulus_summary["duration_minutes"].round(2)
    stimulus_summary.reset_index(drop=True, inplace=True)
    stimulus_summary
else:
    print("No duration records computed.")

if issues:
    pd.DataFrame(
        {"group": list(issues.keys()), "issue": list(issues.values())}
    ).sort_values("group").reset_index(drop=True)

c:\Users\ashra\Documents\NeuralSense\NeuralData\clients\544_WBD_CXCU\analysis\wbdlib\imotions.py:63: DtypeWarning: Columns (3,4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, header=header_rows, **csv_kwargs)
c:\Users\ashra\Documents\NeuralSense\NeuralData\clients\544_WBD_CXCU\analysis\wbdlib\imotions.py:63: DtypeWarning: Columns (3,4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, header=header_rows, **csv_kwargs)
c:\Users\ashra\Documents\NeuralSense\NeuralData\clients\544_WBD_CXCU\analysis\wbdlib\imotions.py:63: DtypeWarning: Columns (3,4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, header=header_rows, **csv_kwargs)
c:\Users\ashra\Documents\NeuralSense\NeuralData\clients\544_WBD_CXCU\analysis\wbdlib\imotions.py:63: DtypeWarning: Columns (3,4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  d

In [19]:
stimulus_summary.head()

,group,stimulus_name,duration_seconds,duration_minutes
0,Group C,07 The Notebook,65.39,1.09
1,Group C,09 I Am Legend - Infected encounter,118.49,1.97
2,Group C,10 The Town - Bank robbery in nun masks,263.21,4.39
3,Group C,Abbott Elementary - S1E9 - Step Class,1291.75,21.53
4,Group C,HOME ALONE,115.09,1.92


In [20]:
issues

{'Group A': 'read_imotions failed: Unable to allocate 853. MiB for an array with shape (210, 532611) and data type float64',
 'Group B': 'read_imotions failed: Unable to allocate 831. MiB for an array with shape (210, 518407) and data type float64',
 'Group D': 'read_imotions failed: Unable to allocate 835. MiB for an array with shape (219, 499863) and data type float64',
 'Group E': 'read_imotions failed: Unable to allocate 825. MiB for an array with shape (212, 510043) and data type float64',
 'Group F': 'read_imotions failed: Unable to allocate 701. MiB for an array with shape (212, 433145) and data type float64'}

In [21]:
stimulus_summary.shape

(6, 4)

In [22]:
stimulus_summary['duration_seconds'].agg(['min','max']).round(2)

min      65.39
max    1291.75
Name: duration_seconds, dtype: float64

In [23]:
stimuli_per_group = stimulus_summary.groupby('group')['stimulus_name'].nunique().reset_index(name='unique_stimuli')
stimuli_per_group

,group,unique_stimuli
0,Group C,6


In [24]:
stimulus_summary.to_csv(project_root / "results" / "stimulus_summary.csv", index=False)

# Feature Extraction

## Stimulus Annotation Overview
- `stimulus_rename` links each group-specific stimulus from `stimulus_summary` to a clean `title` and its presentation `Form` (`Long` or `Short`).
- Some titles appear in both forms; the long cut (Ã¢â€°Ë†30 min) includes the short-form key moment as an embedded segment.
- `key_moments` pinpoints, for every long-form title, when the key moment begins (`Lead-up Duration`) and how long it lasts (`Key moment Duration_LF`).
- These tables let us align short-form clips with the corresponding segment inside the long-form presentation for downstream comparisons.

## Stage 1: Demographics
We extract respondent-level identifiers and timing information from the metadata embedded in each sensor export to seed the unified view (UV). This pass scans every sensor CSV, captures study name, respondent attributes, and recording timestamps, and prepares the foundation for later feature merges.

In [25]:
from wbdlib import (
    derive_respondent_identifier,
    first_segment,
    infer_group_letter,
    parse_recording_timestamp,
 )

metadata_keys = [
    "Study name",
    "Respondent Name",
    "Respondent Group",
    "Recording time"
 ]

sensor_file_paths = sorted(
    (project_root / "data" / "Export").glob("Group */Analyses/*/Sensor Data/*.csv")
 )

demographic_records = []

for csv_path in sensor_file_paths:
    meta = read_imotions_metadata(csv_path, metadata=metadata_keys)

    study_clean = first_segment(meta.get("Study name"))
    respondent_group_clean = first_segment(meta.get("Respondent Group"))
    group_letter = infer_group_letter(study_clean, respondent_group_clean, csv_path)

    respondent_raw = first_segment(meta.get("Respondent Name"))
    respondent_value = derive_respondent_identifier(respondent_raw, csv_path.stem)

    date_study, time_study = parse_recording_timestamp(meta.get("Recording time"))

    demographic_records.append({
        "source_file": csv_path.name,
        "group": group_letter,
        "respondent": respondent_value,
        "date_study": date_study,
        "time_study": time_study
    })

uv_stage1 = pd.DataFrame(demographic_records)

if not uv_stage1.empty:
    uv_stage1 = uv_stage1.sort_values(["group", "respondent"]).reset_index(drop=True)
    uv_stage1["respondent"] = uv_stage1["respondent"].astype(str)

uv = uv_stage1.copy()

uv_stage1

,source_file,group,respondent,date_study,time_study
0,003_104.csv,A,104,10/16/2025,18:09:03
1,002_106.csv,A,106,10/16/2025,19:35:05
2,001_116.csv,A,116,10/18/2025,12:37:40
3,006_14.csv,A,14,10/11/2025,09:32:42
4,007_3.csv,A,3,10/10/2025,09:19:22
...,...,...,...,...,...
78,005_50.csv,F,50,10/14/2025,09:54:03
79,004_60.csv,F,60,10/15/2025,09:34:06
80,003_70.csv,F,70,10/16/2025,09:49:14
81,002_85.csv,F,85,10/17/2025,14:37:41


In [26]:
# Attach supplemental demographics from grid.csv
grid_path = project_root / "data" / "grid.csv"
grid_rename_map = {
    "QB2. Age": "age",
    "QB2. Age.1": "age_group",
    "QA2. Gender": "gender",
    "QC. Ethnicity": "ethnicity",
    "QD. Income": "income_group",
    "Q1. Content Hours Per Week": "content_consumption",
    "Q2. Program Type %- Movies": "content_consumption_movies",
    "Q2. Program Type %- Series": "content_consumption_series",
    "Q2. Program Type %- Short": "content_consumption_short",
    "Comments": "grid_comments",
}

grid_raw_full = pd.read_csv(grid_path, encoding="latin1")
grid_raw_full.columns = [col.strip() for col in grid_raw_full.columns]

id_column = next((col for col in ["respondent", "Respondent", "No.", "No", "Participant", "Participant #"] if col in grid_raw_full.columns), None)
if id_column is None:
    raise ValueError("Unable to locate a respondent identifier column in grid.csv")

grid_raw_full = grid_raw_full.rename(columns={id_column: "respondent"})

available_rename_map = {orig: dest for orig, dest in grid_rename_map.items() if orig in grid_raw_full.columns}
missing_columns = sorted(set(grid_rename_map.keys()) - set(available_rename_map.keys()))
if missing_columns:
    print(f"Warning: The following columns were not found in grid.csv and will be skipped: {missing_columns}")

grid_columns = ["respondent", *available_rename_map.keys()]
grid_raw = grid_raw_full.loc[:, grid_columns].copy()

grid_raw = grid_raw.dropna(subset=["respondent"])

grid_raw["respondent"] = pd.to_numeric(grid_raw["respondent"], errors="coerce")
grid_raw = grid_raw.dropna(subset=["respondent"])
grid_raw["respondent"] = grid_raw["respondent"].astype(int).astype(str)

grid_subset = grid_raw.rename(columns=available_rename_map)

text_cols = ["age_group", "gender", "ethnicity", "income_group", "content_consumption", "grid_comments"]
for col in text_cols:
    if col in grid_subset.columns:
        grid_subset[col] = grid_subset[col].apply(lambda value: value.strip() if isinstance(value, str) else value)

numeric_cols = ["content_consumption_movies", "content_consumption_series", "content_consumption_short", "age"]
for col in numeric_cols:
    if col in grid_subset.columns:
        grid_subset[col] = pd.to_numeric(grid_subset[col], errors="coerce").astype("Int64")

grid_subset = grid_subset.drop_duplicates(subset="respondent", keep="first")

uv_stage1["respondent"] = uv_stage1["respondent"].astype(str).str.strip()
uv_stage1 = uv_stage1.merge(grid_subset, on="respondent", how="left", validate="many_to_one")
uv = uv_stage1.copy()

display_columns = [
    "respondent",
    "age",
    "gender",
    "age_group",
    "ethnicity",
    "income_group",
    "content_consumption",
    "content_consumption_movies",
    "content_consumption_series",
    "content_consumption_short",
    "grid_comments",
]
existing_display_columns = [col for col in display_columns if col in uv_stage1.columns]
uv_stage1.loc[:, existing_display_columns].head()

,respondent,age,gender,age_group,ethnicity,income_group,content_consumption,content_consumption_movies,content_consumption_series,content_consumption_short,grid_comments
0,104,59,Male,44-59,White,"$60,000 or more per year",More than 24 hours per week,10,90,0,NaN
1,106,30,Male,28-43,White,"$60,000 or more per year",3 to 12 hours per week,25,50,25,NaN
2,116,19,Male,18-27,White,"$35,000  $60,000 per year",3 to 12 hours per week,25,50,25,NaN
3,14,33,Male,28-43,Hispanic/Latino/Latina/Latinx,"$60,000 or more per year",More than 24 hours per week,20,40,40,No EEG.
4,3,34,Female,28-43,White,"$60,000 or more per year",12 to 24 hours per week,10,70,20,No EEG.


In [27]:
from wbdlib import build_group_short_long_map

group_short_long_map = build_group_short_long_map()

uv_stage1 = uv_stage1.merge(
    group_short_long_map,
    on="group",
    how="left",
    validate="many_to_one",
)
uv = uv_stage1.copy()

uv_stage1.loc[:, ["group", "respondent", "Short Form", "Long Form"]].head()

,group,respondent,Short Form,Long Form
0,A,104,Mad Max,The Town
1,A,106,Mad Max,The Town
2,A,116,Mad Max,The Town
3,A,14,Mad Max,The Town
4,A,3,Mad Max,The Town


In [28]:
duplicate_respondents = uv_stage1[uv_stage1.duplicated(subset="respondent", keep=False)]
if duplicate_respondents.empty:
    print("No duplicate respondents detected.")
else:
    duplicate_respondents.sort_values("respondent")

No duplicate respondents detected.


In [29]:
uv_stage1.to_csv(project_root / "results" / "uv_stage1.csv", index=False)
uv_stage1

,source_file,group,respondent,date_study,time_study,age,age_group,gender,ethnicity,income_group,content_consumption,content_consumption_movies,content_consumption_series,content_consumption_short,grid_comments,Short Form,Long Form
0,003_104.csv,A,104,10/16/2025,18:09:03,59,44-59,Male,White,"$60,000 or more per year",More than 24 hours per week,10,90,0,NaN,Mad Max,The Town
1,002_106.csv,A,106,10/16/2025,19:35:05,30,28-43,Male,White,"$60,000 or more per year",3 to 12 hours per week,25,50,25,NaN,Mad Max,The Town
2,001_116.csv,A,116,10/18/2025,12:37:40,19,18-27,Male,White,"$35,000  $60,000 per year",3 to 12 hours per week,25,50,25,NaN,Mad Max,The Town
3,006_14.csv,A,14,10/11/2025,09:32:42,33,28-43,Male,Hispanic/Latino/Latina/Latinx,"$60,000 or more per year",More than 24 hours per week,20,40,40,No EEG.,Mad Max,The Town
4,007_3.csv,A,3,10/10/2025,09:19:22,34,28-43,Female,White,"$60,000 or more per year",12 to 24 hours per week,10,70,20,No EEG.,Mad Max,The Town
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,005_50.csv,F,50,10/14/2025,09:54:03,63,60-69,Male,Black/African American,"$60,000 or more per year",12 to 24 hours per week,70,20,10,NaN,Mad Max,Abbot Elementary
79,004_60.csv,F,60,10/15/2025,09:34:06,66,60-69,Male,White,"$35,000  $60,000 per year",3 to 12 hours per week,10,80,10,NaN,Mad Max,Abbot Elementary
80,003_70.csv,F,70,10/16/2025,09:49:14,61,60-69,Female,Black/African American,"$35,000  $60,000 per year",12 to 24 hours per week,30,30,40,NaN,Mad Max,Abbot Elementary
81,002_85.csv,F,85,10/17/2025,14:37:41,34,28-43,Female,White,"$60,000 or more per year",12 to 24 hours per week,40,60,0,NaN,Mad Max,Abbot Elementary


## Stage 2: Survey Data

### Overview
Stage 2 ingests the exposure-day survey responses while treating the Stage 1 export as the immutable base. Respondent metadata (age, gender, group, and the long/short assignment) is always reloaded from `results/uv_stage1.csv` before any survey joins. The section engineers Likert scores, familiarity composites, and open-ended extracts so downstream modeling can combine perceptual metrics with the demographic scaffold without relying on sensor features.


### Workflow Summary
1. Reload `results/uv_stage1.csv` to obtain the canonical respondent roster and long/short stimulus assignment.
2. Build the survey metadata lookup (question map, polarity, subscales) to drive scoring logic.
3. Ingest every group-level `MERGED_SURVEY_RESPONSE_MATRIX` export, harmonize respondent IDs, and apply the rename map.
4. Score Likert, familiarity, and recency responses; compute enjoyment composites; and archive open-ended text separately.
5. Merge the numeric survey features onto the Stage 1 base, logging missing or duplicate respondents to `uv_stage2_issues.csv`.
6. Write Stage 2 deliverables: `uv_stage2_features.csv`, `uv_stage2_open_ended.csv`, and the combined `uv_stage2.csv`.


In [13]:
stage1_path = project_root / "results" / "uv_stage1.csv"
if not stage1_path.exists():
    raise FileNotFoundError(f"Stage 1 output not found at {stage1_path}.")
uv_stage1_base = pd.read_csv(stage1_path)
uv_stage1_base["respondent"] = uv_stage1_base["respondent"].astype(str).str.strip()
uv_stage1 = uv_stage1_base.copy()
uv_stage1_base.shape
uv_stage1_base.head()


,source_file,group,respondent,date_study,time_study,age,age_group,gender,ethnicity,income_group,content_consumption,content_consumption_movies,content_consumption_series,content_consumption_short,grid_comments,Short Form,Long Form
0,003_104.csv,A,104,10/16/2025,18:09:03,59,44-59,Male,White,"$60,000 or more per year",More than 24 hours per week,10,90,0,NaN,Mad Max,The Town
1,002_106.csv,A,106,10/16/2025,19:35:05,30,28-43,Male,White,"$60,000 or more per year",3 to 12 hours per week,25,50,25,NaN,Mad Max,The Town
2,001_116.csv,A,116,10/18/2025,12:37:40,19,18-27,Male,White,"$35,000  $60,000 per year",3 to 12 hours per week,25,50,25,NaN,Mad Max,The Town
3,006_14.csv,A,14,10/11/2025,09:32:42,33,28-43,Male,Hispanic/Latino/Latina/Latinx,"$60,000 or more per year",More than 24 hours per week,20,40,40,No EEG.,Mad Max,The Town
4,007_3.csv,A,3,10/10/2025,09:19:22,34,28-43,Female,White,"$60,000 or more per year",12 to 24 hours per week,10,70,20,No EEG.,Mad Max,The Town


In [14]:
import csv
from collections import defaultdict
import re

survey_rename_map = pd.read_csv(project_root / "data" / "survey_column_rename_stage3.csv")
survey_questions = pd.read_csv(project_root / "data" / "survey_questions.csv")

survey_questions["question_code"] = survey_questions["question_code"].astype(str).str.strip()
survey_questions["question_type"] = survey_questions["question_type"].str.lower()
survey_questions["subscale"] = survey_questions["subscale"].fillna("")
survey_questions["polarity"] = survey_questions["polarity"].fillna("")

survey_metadata = (
    survey_rename_map
    .merge(survey_questions, on="question_code", how="left", suffixes=("", "_details"))
)

survey_metadata["question_type"] = survey_metadata["question_type"].fillna("likert")
survey_metadata["subscale"] = survey_metadata["subscale"].fillna("")
survey_metadata["polarity"] = survey_metadata["polarity"].fillna("")
survey_metadata_lookup = (
    survey_metadata
    .drop_duplicates(subset=["target_column"])
    .set_index("target_column")
)

survey_files = sorted(
    (project_root / "data" / "Export").glob("Group */Analyses/*/Survey/MERGED_SURVEY_RESPONSE_MATRIX-*.txt")
)

if not survey_files:
    raise FileNotFoundError("No survey response text files detected under data/Export/*/Survey/.")

len(survey_files)

12

In [15]:
from wbdlib import (
    clean_response,
    clip_zero_to_four,
    parse_likert_value,
    reverse_likert,
    safe_write_csv,
    score_familiarity,
    score_last_watched,
 )

In [16]:
from wbdlib import (
    extract_group_letter_from_path,
    load_survey_file,
    rename_survey_columns,
 )


survey_frames = []

for survey_path in survey_files:
    group_letter = extract_group_letter_from_path(survey_path)
    df = load_survey_file(survey_path)
    df.columns = [col.strip() for col in df.columns]
    df = df.replace({"EMPTY FIELD": np.nan})
    df["RESPONDENT"] = df["RESPONDENT"].astype(str).str.strip()
    df = df.rename(
        columns={
            "RESPONDENT": "respondent",
            "GROUP": "survey_group",
            "STUDY": "survey_study",
            "GENDER": "survey_gender",
            "AGE": "survey_age",
        }
    )
    if "survey_group" not in df.columns:
        df["survey_group"] = group_letter
    df["survey_group"] = df["survey_group"].fillna(group_letter).astype(str).str.strip()
    if "survey_study" in df.columns:
        df["survey_study"] = df["survey_study"].astype(str).str.strip()
    else:
        df["survey_study"] = np.nan
    if "survey_gender" in df.columns:
        df["survey_gender"] = df["survey_gender"].astype(str).str.strip()
    else:
        df["survey_gender"] = np.nan
    df["survey_file"] = survey_path.name
    if "survey_age" in df.columns:
        df["survey_age"] = pd.to_numeric(df["survey_age"], errors="coerce")
    else:
        df["survey_age"] = np.nan
    base_columns = df[
        [
            "respondent",
            "survey_group",
            "survey_study",
            "survey_gender",
            "survey_age",
            "survey_file",
        ]
    ].copy()
    renamed_columns = rename_survey_columns(df, survey_metadata, group_letter)
    df_prepared = pd.concat([base_columns, renamed_columns], axis=1)
    survey_frames.append(df_prepared)

survey_responses = pd.concat(survey_frames, ignore_index=True)
survey_responses

C:\Users\ashra\AppData\Local\Temp\ipykernel_36856\2064908789.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({"EMPTY FIELD": np.nan})
C:\Users\ashra\AppData\Local\Temp\ipykernel_36856\2064908789.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({"EMPTY FIELD": np.nan})
C:\Users\ashra\AppData\Local\Temp\ipykernel_36856\2064908789.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.in

,respondent,survey_group,survey_study,survey_gender,survey_age,survey_file,Long_The Town_Survey_Enjoyment_E1,Long_The Town_Survey_Enjoyment_E2,Long_The Town_Survey_Enjoyment_E3,Long_The Town_Survey_Enjoyment_E4,...,Short_Abbot Elementary_Survey_Enjoyment_WBD1,Short_Abbot Elementary_Survey_Enjoyment_WBD2,Short_Abbot Elementary_Survey_Enjoyment_WBD3,Short_Abbot Elementary_Survey_Enjoyment_WBD4,Short_Abbot Elementary_Survey_Enjoyment_WBD5,Short_Abbot Elementary_Survey_Enjoyment_E21,Short_Abbot Elementary_Survey_Enjoyment_E22,Short_Abbot Elementary_Survey_Familiarity_F1,Short_Abbot Elementary_Survey_Familiarity_F2,Short_Abbot Elementary_Survey_Familiarity_F3
0,83,Default,Group A,MALE,69,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,5 = Strongly Agree,5 = Strongly Agree,4 = Agree,5 = Strongly Agree,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,81,Default,Group A,FEMALE,24,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,5 = Strongly Agree,4 = Agree,4 = Agree,5 = Strongly Agree,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,99,Default,Group A,FEMALE,25,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,4 = Agree,4 = Agree,3 = Neither Agree nor Disagree,4 = Agree,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,52,Default,Group A,FEMALE,50,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,5 = Strongly Agree,5 = Strongly Agree,4 = Agree,4 = Agree,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8,Default,Group A,OTHER,51,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,5 = Strongly Agree,5 = Strongly Agree,1 = Strongly Disagree,5 = Strongly Agree,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,85,Default,Group F,FEMALE,34,MERGED_SURVEY_RESPONSE_MATRIX-F2.txt,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78,70,Default,Group F,FEMALE,61,MERGED_SURVEY_RESPONSE_MATRIX-F2.txt,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,96,Default,Group F,FEMALE,29,MERGED_SURVEY_RESPONSE_MATRIX-F2.txt,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,41,Default,Group F,FEMALE,53,MERGED_SURVEY_RESPONSE_MATRIX-F2.txt,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
survey_responses["respondent"] = survey_responses["respondent"].astype(str).str.strip()

duplicate_ids = sorted(survey_responses.loc[survey_responses["respondent"].duplicated(), "respondent"].unique())
if duplicate_ids:
    print(f"Warning: duplicate survey rows detected for respondents: {duplicate_ids}")

survey_numeric = survey_responses.drop_duplicates(subset=["respondent"], keep="first").copy()
survey_numeric

,respondent,survey_group,survey_study,survey_gender,survey_age,survey_file,Long_The Town_Survey_Enjoyment_E1,Long_The Town_Survey_Enjoyment_E2,Long_The Town_Survey_Enjoyment_E3,Long_The Town_Survey_Enjoyment_E4,...,Short_Abbot Elementary_Survey_Enjoyment_WBD1,Short_Abbot Elementary_Survey_Enjoyment_WBD2,Short_Abbot Elementary_Survey_Enjoyment_WBD3,Short_Abbot Elementary_Survey_Enjoyment_WBD4,Short_Abbot Elementary_Survey_Enjoyment_WBD5,Short_Abbot Elementary_Survey_Enjoyment_E21,Short_Abbot Elementary_Survey_Enjoyment_E22,Short_Abbot Elementary_Survey_Familiarity_F1,Short_Abbot Elementary_Survey_Familiarity_F2,Short_Abbot Elementary_Survey_Familiarity_F3
0,83,Default,Group A,MALE,69,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,5 = Strongly Agree,5 = Strongly Agree,4 = Agree,5 = Strongly Agree,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,81,Default,Group A,FEMALE,24,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,5 = Strongly Agree,4 = Agree,4 = Agree,5 = Strongly Agree,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,99,Default,Group A,FEMALE,25,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,4 = Agree,4 = Agree,3 = Neither Agree nor Disagree,4 = Agree,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,52,Default,Group A,FEMALE,50,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,5 = Strongly Agree,5 = Strongly Agree,4 = Agree,4 = Agree,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8,Default,Group A,OTHER,51,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,5 = Strongly Agree,5 = Strongly Agree,1 = Strongly Disagree,5 = Strongly Agree,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,85,Default,Group F,FEMALE,34,MERGED_SURVEY_RESPONSE_MATRIX-F2.txt,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78,70,Default,Group F,FEMALE,61,MERGED_SURVEY_RESPONSE_MATRIX-F2.txt,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,96,Default,Group F,FEMALE,29,MERGED_SURVEY_RESPONSE_MATRIX-F2.txt,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,41,Default,Group F,FEMALE,53,MERGED_SURVEY_RESPONSE_MATRIX-F2.txt,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
for column, meta in survey_metadata_lookup.iterrows():
    if column not in survey_numeric.columns:
        continue

    question_code = (meta.get("question_code") or "").strip().upper()
    question_type = (meta.get("question_type") or "").strip().lower()
    polarity = (meta.get("polarity") or "").strip().lower()

    if question_code in {"F1", "F3"} or column.endswith("_Survey_Familiarity_F1") or column.endswith("_Survey_Familiarity_F3"):
        survey_numeric[column] = survey_numeric[column].apply(score_familiarity)
    elif question_code == "F2" or column.endswith("_Survey_Familiarity_F2"):
        survey_numeric[column] = survey_numeric[column].apply(score_last_watched)
    elif question_type == "likert":
        survey_numeric[column] = survey_numeric[column].apply(parse_likert_value)
        if polarity == "negative":
            survey_numeric[column] = survey_numeric[column].apply(reverse_likert)

familiarity_columns = [
    column
    for column in survey_numeric.columns
    if column.endswith("_Survey_Familiarity_F1")
    or column.endswith("_Survey_Familiarity_F2")
    or column.endswith("_Survey_Familiarity_F3")
]

for column in familiarity_columns:
    survey_numeric[column] = survey_numeric[column].apply(clip_zero_to_four)

survey_numeric

,respondent,survey_group,survey_study,survey_gender,survey_age,survey_file,Long_The Town_Survey_Enjoyment_E1,Long_The Town_Survey_Enjoyment_E2,Long_The Town_Survey_Enjoyment_E3,Long_The Town_Survey_Enjoyment_E4,...,Short_Abbot Elementary_Survey_Enjoyment_WBD1,Short_Abbot Elementary_Survey_Enjoyment_WBD2,Short_Abbot Elementary_Survey_Enjoyment_WBD3,Short_Abbot Elementary_Survey_Enjoyment_WBD4,Short_Abbot Elementary_Survey_Enjoyment_WBD5,Short_Abbot Elementary_Survey_Enjoyment_E21,Short_Abbot Elementary_Survey_Enjoyment_E22,Short_Abbot Elementary_Survey_Familiarity_F1,Short_Abbot Elementary_Survey_Familiarity_F2,Short_Abbot Elementary_Survey_Familiarity_F3
0,83,Default,Group A,MALE,69,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,5.0,5.0,4.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,81,Default,Group A,FEMALE,24,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,5.0,4.0,4.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,99,Default,Group A,FEMALE,25,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,4.0,4.0,3.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,52,Default,Group A,FEMALE,50,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,5.0,5.0,4.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8,Default,Group A,OTHER,51,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,5.0,5.0,1.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,85,Default,Group F,FEMALE,34,MERGED_SURVEY_RESPONSE_MATRIX-F2.txt,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78,70,Default,Group F,FEMALE,61,MERGED_SURVEY_RESPONSE_MATRIX-F2.txt,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,96,Default,Group F,FEMALE,29,MERGED_SURVEY_RESPONSE_MATRIX-F2.txt,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,41,Default,Group F,FEMALE,53,MERGED_SURVEY_RESPONSE_MATRIX-F2.txt,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
subscale_columns = defaultdict(list)
overall_enjoyment_columns = defaultdict(list)

for column, meta in survey_metadata_lookup.iterrows():
    if column not in survey_numeric.columns:
        continue
    if meta.get("topic") != "enjoyment":
        continue
    if (meta.get("question_type") or "").strip().lower() != "likert":
        continue
    prefix = column.split("_Survey_")[0]
    subscale = (meta.get("subscale") or "").strip()
    overall_enjoyment_columns[prefix].append(column)
    if subscale:
        subscale_columns[(prefix, subscale)].append(column)

for (prefix, subscale), cols in subscale_columns.items():
    values = survey_numeric[cols]
    sum_col = f"{prefix}_Survey_{subscale}_Sum"
    count_col = f"{prefix}_Survey_{subscale}_Count"
    mean_col = f"{prefix}_Survey_{subscale}_Mean"
    norm_col = f"{prefix}_Survey_{subscale}_Normalized"
    count_values = values.notna().sum(axis=1)
    sum_values = values.sum(axis=1, min_count=1)
    survey_numeric[count_col] = count_values
    survey_numeric[sum_col] = sum_values
    survey_numeric[mean_col] = np.where(count_values > 0, sum_values / count_values, np.nan)
    survey_numeric[norm_col] = np.where(
        count_values > 0,
        np.clip((sum_values - count_values) / (4.0 * count_values), 0, 1),
        np.nan,
    )

for prefix, cols in overall_enjoyment_columns.items():
    values = survey_numeric[cols]
    sum_col = f"{prefix}_Survey_EnjoymentComposite_Sum"
    count_col = f"{prefix}_Survey_EnjoymentComposite_Count"
    mean_col = f"{prefix}_Survey_EnjoymentComposite_Mean"
    norm_col = f"{prefix}_Survey_EnjoymentComposite_Normalized"
    norm_corrected_col = f"{prefix}_Survey_EnjoymentComposite_NormalizedCorrected"
    corrected_col = f"{prefix}_Survey_EnjoymentComposite_Corrected"
    count_values = values.notna().sum(axis=1)

    raw_components = pd.DataFrame(index=values.index, dtype=float)
    corrected_components = pd.DataFrame(index=values.index, dtype=float)

    for column in cols:
        polarity_meta = ""
        if column in survey_metadata_lookup.index:
            polarity_meta = (survey_metadata_lookup.loc[column].get("polarity") or "").strip().lower()
        if column in survey_responses.columns:
            raw_series = survey_responses.loc[values.index, column]
            parsed_series = raw_series.apply(parse_likert_value)
        else:
            fallback_series = pd.to_numeric(survey_numeric.loc[values.index, column], errors="coerce")
            if polarity_meta == "negative":
                parsed_series = fallback_series.apply(reverse_likert)
            else:
                parsed_series = fallback_series
        raw_components[column] = pd.to_numeric(parsed_series, errors="coerce")
        corrected_series = pd.to_numeric(parsed_series, errors="coerce")
        if polarity_meta == "negative":
            corrected_series = corrected_series.apply(reverse_likert)
        corrected_components[column] = corrected_series

    enjoyment_questions = ['E1','E2','E3','E4','E5','E6','E7','E8','E9','E10','E11','E12','E13','E14','E15','E16','E17','E18','E19','E20','E21','E22','E23','E24','E25']
    keep_cols = [c for c in raw_components.columns if any(eq in c for eq in enjoyment_questions)]
    raw_components = raw_components[keep_cols]
    corrected_components = corrected_components[keep_cols]
    raw_sum_values = raw_components.sum(axis=1, min_count=1)
    corrected_sum = corrected_components.sum(axis=1, min_count=1)
    raw_normalized = np.where(
        count_values > 0,
        np.clip((raw_sum_values - count_values) / (4.0 * count_values), 0, 1),
        np.nan,
    )
    corrected_normalized = np.where(
        count_values > 0,
        np.clip((corrected_sum - count_values) / (4.0 * count_values), 0, 1),
        np.nan,
    )
    corrected_mean = np.where(count_values > 0, corrected_sum / count_values, np.nan)

    survey_numeric[count_col] = count_values
    survey_numeric[sum_col] = raw_sum_values
    survey_numeric[corrected_col] = corrected_sum
    survey_numeric[mean_col] = corrected_mean
    survey_numeric[norm_col] = raw_normalized
    survey_numeric[norm_corrected_col] = corrected_normalized

survey_numeric

C:\Users\ashra\AppData\Local\Temp\ipykernel_36856\865435719.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  survey_numeric[count_col] = count_values
C:\Users\ashra\AppData\Local\Temp\ipykernel_36856\865435719.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  survey_numeric[sum_col] = sum_values
C:\Users\ashra\AppData\Local\Temp\ipykernel_36856\865435719.py:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining

,respondent,survey_group,survey_study,survey_gender,survey_age,survey_file,Long_The Town_Survey_Enjoyment_E1,Long_The Town_Survey_Enjoyment_E2,Long_The Town_Survey_Enjoyment_E3,Long_The Town_Survey_Enjoyment_E4,...,Long_Abbot Elementary_Survey_EnjoymentComposite_Corrected,Long_Abbot Elementary_Survey_EnjoymentComposite_Mean,Long_Abbot Elementary_Survey_EnjoymentComposite_Normalized,Long_Abbot Elementary_Survey_EnjoymentComposite_NormalizedCorrected,Short_Abbot Elementary_Survey_EnjoymentComposite_Count,Short_Abbot Elementary_Survey_EnjoymentComposite_Sum,Short_Abbot Elementary_Survey_EnjoymentComposite_Corrected,Short_Abbot Elementary_Survey_EnjoymentComposite_Mean,Short_Abbot Elementary_Survey_EnjoymentComposite_Normalized,Short_Abbot Elementary_Survey_EnjoymentComposite_NormalizedCorrected
0,83,Default,Group A,MALE,69,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,5.0,5.0,4.0,5.0,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
1,81,Default,Group A,FEMALE,24,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,5.0,4.0,4.0,5.0,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
2,99,Default,Group A,FEMALE,25,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,4.0,4.0,3.0,4.0,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
3,52,Default,Group A,FEMALE,50,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,5.0,5.0,4.0,4.0,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
4,8,Default,Group A,OTHER,51,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,5.0,5.0,1.0,5.0,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,85,Default,Group F,FEMALE,34,MERGED_SURVEY_RESPONSE_MATRIX-F2.txt,NaN,NaN,NaN,NaN,...,43.0,3.583333,0.687500,0.645833,0,NaN,NaN,NaN,NaN,NaN
78,70,Default,Group F,FEMALE,61,MERGED_SURVEY_RESPONSE_MATRIX-F2.txt,NaN,NaN,NaN,NaN,...,59.0,4.916667,0.895833,0.979167,0,NaN,NaN,NaN,NaN,NaN
79,96,Default,Group F,FEMALE,29,MERGED_SURVEY_RESPONSE_MATRIX-F2.txt,NaN,NaN,NaN,NaN,...,53.0,4.416667,0.770833,0.854167,0,NaN,NaN,NaN,NaN,NaN
80,41,Default,Group F,FEMALE,53,MERGED_SURVEY_RESPONSE_MATRIX-F2.txt,NaN,NaN,NaN,NaN,...,47.0,3.916667,0.645833,0.729167,0,NaN,NaN,NaN,NaN,NaN


In [20]:
familiarity_prefixes = sorted({
    column.split("_Survey_")[0]
    for column in survey_numeric.columns
    if column.endswith("_Survey_Familiarity_F1")
})

for prefix in familiarity_prefixes:
    f1_col = f"{prefix}_Survey_Familiarity_F1"
    f2_col = f"{prefix}_Survey_Familiarity_F2"
    if f1_col not in survey_numeric.columns or f2_col not in survey_numeric.columns:
        continue
    c1_col = f"{prefix}_Survey_Familiarity_C1"
    count_col = f"{prefix}_Survey_Familiarity_C1_Count"
    norm_col = f"{prefix}_Survey_Familiarity_C1_Normalized"
    pair = survey_numeric[[f1_col, f2_col]]
    sum_values = pair.fillna(0).sum(axis=1)
    count_values = pair.notna().sum(axis=1)
    survey_numeric[c1_col] = sum_values
    survey_numeric[count_col] = count_values
    survey_numeric[norm_col] = np.where(
        count_values > 0,
        np.clip(sum_values / (4 * count_values), 0, 1),
        np.nan,
    )

survey_numeric

C:\Users\ashra\AppData\Local\Temp\ipykernel_36856\4132188637.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  survey_numeric[c1_col] = sum_values
C:\Users\ashra\AppData\Local\Temp\ipykernel_36856\4132188637.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  survey_numeric[count_col] = count_values
C:\Users\ashra\AppData\Local\Temp\ipykernel_36856\4132188637.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joini

,respondent,survey_group,survey_study,survey_gender,survey_age,survey_file,Long_The Town_Survey_Enjoyment_E1,Long_The Town_Survey_Enjoyment_E2,Long_The Town_Survey_Enjoyment_E3,Long_The Town_Survey_Enjoyment_E4,...,Long_The Town_Survey_Familiarity_C1_Normalized,Short_Abbot Elementary_Survey_Familiarity_C1,Short_Abbot Elementary_Survey_Familiarity_C1_Count,Short_Abbot Elementary_Survey_Familiarity_C1_Normalized,Short_Mad Max_Survey_Familiarity_C1,Short_Mad Max_Survey_Familiarity_C1_Count,Short_Mad Max_Survey_Familiarity_C1_Normalized,Short_The Town_Survey_Familiarity_C1,Short_The Town_Survey_Familiarity_C1_Count,Short_The Town_Survey_Familiarity_C1_Normalized
0,83,Default,Group A,MALE,69,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,5.0,5.0,4.0,5.0,...,0.000,0.0,0,NaN,0.0,2,0.000,0.0,0,NaN
1,81,Default,Group A,FEMALE,24,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,5.0,4.0,4.0,5.0,...,0.750,0.0,0,NaN,4.0,2,0.500,0.0,0,NaN
2,99,Default,Group A,FEMALE,25,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,4.0,4.0,3.0,4.0,...,0.000,0.0,0,NaN,1.0,2,0.125,0.0,0,NaN
3,52,Default,Group A,FEMALE,50,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,5.0,5.0,4.0,4.0,...,0.000,0.0,0,NaN,0.0,2,0.000,0.0,0,NaN
4,8,Default,Group A,OTHER,51,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,5.0,5.0,1.0,5.0,...,0.125,0.0,0,NaN,3.0,2,0.375,0.0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,85,Default,Group F,FEMALE,34,MERGED_SURVEY_RESPONSE_MATRIX-F2.txt,NaN,NaN,NaN,NaN,...,NaN,0.0,0,NaN,5.0,2,0.625,0.0,0,NaN
78,70,Default,Group F,FEMALE,61,MERGED_SURVEY_RESPONSE_MATRIX-F2.txt,NaN,NaN,NaN,NaN,...,NaN,0.0,0,NaN,0.0,2,0.000,0.0,0,NaN
79,96,Default,Group F,FEMALE,29,MERGED_SURVEY_RESPONSE_MATRIX-F2.txt,NaN,NaN,NaN,NaN,...,NaN,0.0,0,NaN,3.0,2,0.375,0.0,0,NaN
80,41,Default,Group F,FEMALE,53,MERGED_SURVEY_RESPONSE_MATRIX-F2.txt,NaN,NaN,NaN,NaN,...,NaN,0.0,0,NaN,0.0,2,0.000,0.0,0,NaN


In [21]:
open_ended_columns = [
    column
    for column, meta in survey_metadata_lookup.iterrows()
    if column in survey_numeric.columns and (meta.get("question_type") or "").strip().lower() == "open ended"
]

survey_open_ended = survey_responses[[
    "respondent",
    "survey_group",
    "survey_study",
    "survey_gender",
    "survey_age",
    "survey_file",
    *open_ended_columns,
]].copy()

survey_features = survey_numeric.drop(columns=[col for col in open_ended_columns if col in survey_numeric.columns])

# Integrate screening familiarity composites
screening_path = project_root / "results" / "individual_composite_scores.csv"
if screening_path.exists():
    screening_raw = pd.read_csv(screening_path)
    screening_raw["respondent"] = screening_raw["respondent"].astype(str).str.strip()

    screening_value_columns = [
        col
        for col in screening_raw.columns
        if col.endswith("_Survey_Familiarity_F1")
        or col.endswith("_Survey_Familiarity_F2")
        or col.endswith("_Survey_Familiarity_F3")
        or col.endswith("_Survey_Familiarity_C1")
    ]

    respondent_groups = (
        uv_stage1
        .loc[:, ["respondent", "group"]]
        .assign(group=lambda df: df["group"].astype(str).str.strip().str.upper())
        .set_index("respondent")
        .to_dict()
    )["group"]

    title_normalization = {
        "mad max fury road": "Mad Max",
        "mad max": "Mad Max",
        "the town": "The Town",
        "abbot elementary": "Abbot Elementary",
        "abbott elementary": "Abbot Elementary",
    }

    def canonicalize_title(raw_title: str) -> str:
        cleaned = str(raw_title).strip()
        return title_normalization.get(cleaned.lower(), cleaned)

    stimulus_map = pd.read_csv(project_root / "data" / "stimulus_rename.csv")
    stimulus_map["group_letter"] = stimulus_map["group"].str.extract(r"([A-F])", expand=False)
    stimulus_map["title_clean"] = stimulus_map["title"].astype(str).str.strip()

    group_title_form_lookup = {}
    default_form_per_title = {}

    for row in stimulus_map.itertuples():
        if pd.isna(row.group_letter) or pd.isna(row.title_clean) or pd.isna(row.form):
            continue
        canonical_title = canonicalize_title(row.title_clean)
        form_value = str(row.form).title()
        group_title_form_lookup[(row.group_letter, canonical_title)] = form_value
        default_form_per_title.setdefault(canonical_title, form_value)

    screening_records = []

    for _, row in screening_raw.iterrows():
        respondent_id = row.get("respondent")
        if respondent_id is None:
            continue
        respondent_id = str(respondent_id).strip()
        group_letter = respondent_groups.get(respondent_id)
        for column in screening_value_columns:
            value = row.get(column)
            if pd.isna(value) or value == "":
                continue
            base_part, _, suffix_part = column.partition("_Survey_Familiarity_")
            if not suffix_part:
                continue
            question_code = suffix_part.strip()
            canonical_title = canonicalize_title(base_part.strip())
            form_value = None
            if group_letter:
                form_value = group_title_form_lookup.get((group_letter, canonical_title))
            if form_value is None:
                form_value = default_form_per_title.get(canonical_title, "Long")
            target_column = f"{form_value}_{canonical_title}_Screening_Familiarity_{question_code}"
            screening_records.append({
                "respondent": respondent_id,
                "target_column": target_column,
                "value": pd.to_numeric(value, errors="coerce")
            })

    if screening_records:
        screening_features = (
            pd.DataFrame(screening_records)
            .pivot_table(index="respondent", columns="target_column", values="value", aggfunc="first")
            .reset_index()
        )
        screening_features.columns.name = None
        survey_features = survey_features.merge(screening_features, on="respondent", how="left")

survey_features

,respondent,survey_group,survey_study,survey_gender,survey_age,survey_file,Long_The Town_Survey_Enjoyment_E1,Long_The Town_Survey_Enjoyment_E2,Long_The Town_Survey_Enjoyment_E3,Long_The Town_Survey_Enjoyment_E4,...,Long_The Town_Screening_Familiarity_F2,Short_Abbot Elementary_Screening_Familiarity_C1,Short_Abbot Elementary_Screening_Familiarity_F1,Short_Abbot Elementary_Screening_Familiarity_F2,Short_Mad Max_Screening_Familiarity_C1,Short_Mad Max_Screening_Familiarity_F1,Short_Mad Max_Screening_Familiarity_F2,Short_The Town_Screening_Familiarity_C1,Short_The Town_Screening_Familiarity_F1,Short_The Town_Screening_Familiarity_F2
0,83,Default,Group A,MALE,69,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,5.0,5.0,4.0,5.0,...,NaN,5.0,4.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN
1,81,Default,Group A,FEMALE,24,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,5.0,4.0,4.0,5.0,...,2.0,1.0,1.0,NaN,4.0,3.0,1.0,NaN,NaN,NaN
2,99,Default,Group A,FEMALE,25,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,4.0,4.0,3.0,4.0,...,NaN,2.0,2.0,NaN,4.0,3.0,1.0,NaN,NaN,NaN
3,52,Default,Group A,FEMALE,50,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,5.0,5.0,4.0,4.0,...,NaN,1.0,1.0,NaN,2.0,2.0,NaN,NaN,NaN,NaN
4,8,Default,Group A,OTHER,51,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,5.0,5.0,1.0,5.0,...,1.0,4.0,3.0,1.0,4.0,3.0,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,85,Default,Group F,FEMALE,34,MERGED_SURVEY_RESPONSE_MATRIX-F2.txt,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4.0,3.0,1.0,0.0,0.0,NaN
78,70,Default,Group F,FEMALE,61,MERGED_SURVEY_RESPONSE_MATRIX-F2.txt,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1.0,1.0,NaN
79,96,Default,Group F,FEMALE,29,MERGED_SURVEY_RESPONSE_MATRIX-F2.txt,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4.0,3.0,1.0,4.0,3.0,1.0
80,41,Default,Group F,FEMALE,53,MERGED_SURVEY_RESPONSE_MATRIX-F2.txt,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4.0,3.0,1.0,4.0,3.0,1.0


In [22]:
results_dir = project_root / "results"
results_dir.mkdir(parents=True, exist_ok=True)

stage1_path = results_dir / "uv_stage1.csv"
if not stage1_path.exists():
    raise FileNotFoundError(f"Stage 1 output not found at {stage1_path}.")
uv_stage1_base = pd.read_csv(stage1_path)
uv_stage1_base["respondent"] = uv_stage1_base["respondent"].astype(str).str.strip()

survey_features = survey_features.copy()
survey_features["respondent"] = survey_features["respondent"].astype(str).str.strip()

uv_stage2 = uv_stage1_base.merge(survey_features, on="respondent", how="left")

issues_records: list[dict] = []
feature_ids = survey_features["respondent"].dropna().astype(str).str.strip()
expected_ids = uv_stage1_base["respondent"].astype(str).str.strip()

missing_ids = sorted(set(expected_ids) - set(feature_ids))
for respondent in missing_ids:
    issues_records.append({
        "respondent": respondent,
        "issue": "stage2_features_missing",
    })

duplicate_mask = feature_ids.duplicated(keep=False)
if duplicate_mask.any():
    duplicates = feature_ids[duplicate_mask]
    for respondent in sorted(duplicates.unique()):
        count = int((feature_ids == respondent).sum())
        issues_records.append({
            "respondent": respondent,
            "issue": f"stage2_duplicate_records_count_{count}",
        })

issues_df = pd.DataFrame(issues_records)

features_path = safe_write_csv(survey_features, results_dir / "uv_stage2_features.csv")
open_ended_path = safe_write_csv(survey_open_ended, results_dir / "uv_stage2_open_ended.csv")
uv_path = safe_write_csv(uv_stage2, results_dir / "uv_stage2.csv")

issues_path = results_dir / "uv_stage2_issues.csv"
if issues_df.empty:
    if issues_path.exists():
        issues_path.unlink()
    print("No Stage 2 survey issues detected.")
else:
    safe_write_csv(issues_df, issues_path)
    print(f"Logged {issues_df.shape[0]} Stage 2 issues to {issues_path.name}.")

print(
    f"Stage 2 survey features saved to {features_path.name} with {survey_features.shape[0]} respondents and "
    f"{survey_features.shape[1] - 1} feature columns."
)
print(f"Open-ended responses archived to {open_ended_path.name}.")
print(f"Unified view with Stage 2 survey data exported to {uv_path.name}.")

uv_stage2

Logged 1 Stage 2 issues to uv_stage2_issues.csv.
Stage 2 survey features saved to uv_stage2_features.csv with 82 respondents and 365 feature columns.
Open-ended responses archived to uv_stage2_open_ended.csv.
Unified view with Stage 2 survey data exported to uv_stage2.csv.


,source_file,group,respondent,date_study,time_study,age,age_group,gender,ethnicity,income_group,...,Long_The Town_Screening_Familiarity_F2,Short_Abbot Elementary_Screening_Familiarity_C1,Short_Abbot Elementary_Screening_Familiarity_F1,Short_Abbot Elementary_Screening_Familiarity_F2,Short_Mad Max_Screening_Familiarity_C1,Short_Mad Max_Screening_Familiarity_F1,Short_Mad Max_Screening_Familiarity_F2,Short_The Town_Screening_Familiarity_C1,Short_The Town_Screening_Familiarity_F1,Short_The Town_Screening_Familiarity_F2
0,003_104.csv,A,104,10/16/2025,18:09:03,59,44-59,Male,White,"$60,000 or more per year",...,1.0,0.0,0.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN
1,002_106.csv,A,106,10/16/2025,19:35:05,30,28-43,Male,White,"$60,000 or more per year",...,NaN,4.0,3.0,1.0,4.0,3.0,1.0,NaN,NaN,NaN
2,001_116.csv,A,116,10/18/2025,12:37:40,19,18-27,Male,White,"$35,000  $60,000 per year",...,NaN,1.0,1.0,NaN,4.0,3.0,1.0,NaN,NaN,NaN
3,006_14.csv,A,14,10/11/2025,09:32:42,33,28-43,Male,Hispanic/Latino/Latina/Latinx,"$60,000 or more per year",...,1.0,8.0,4.0,4.0,8.0,4.0,4.0,NaN,NaN,NaN
4,007_3.csv,A,3,10/10/2025,09:19:22,34,28-43,Female,White,"$60,000 or more per year",...,NaN,1.0,1.0,NaN,5.0,3.0,2.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,005_50.csv,F,50,10/14/2025,09:54:03,63,60-69,Male,Black/African American,"$60,000 or more per year",...,NaN,NaN,NaN,NaN,6.0,4.0,2.0,6.0,3.0,3.0
79,004_60.csv,F,60,10/15/2025,09:34:06,66,60-69,Male,White,"$35,000  $60,000 per year",...,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1.0,1.0,NaN
80,003_70.csv,F,70,10/16/2025,09:49:14,61,60-69,Female,Black/African American,"$35,000  $60,000 per year",...,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1.0,1.0,NaN
81,002_85.csv,F,85,10/17/2025,14:37:41,34,28-43,Female,White,"$60,000 or more per year",...,NaN,NaN,NaN,NaN,4.0,3.0,1.0,0.0,0.0,NaN


### Stage 2 Long-Form Export
Transforms the Stage 2 survey outputs into a respondent-by-question table with metadata joins and derived score rows so downstream work can consume a single tidy feed.

In [23]:
from pathlib import Path
import numpy as np
import pandas as pd
import re

DERIVED_PREFIX = "DERIVED_STAGE2_"
feature_pattern = re.compile(r"^(Long|Short)_(.*?)_(Survey|Screening)_(.*)$")


def load_or_read(frame_name: str, path: Path) -> pd.DataFrame:
    existing = globals().get(frame_name)
    if existing is not None:
        return existing.copy()
    if not path.exists():
        raise FileNotFoundError(f"Required artifact {path.name} not found at {path}.")
    return pd.read_csv(path)


def standardize_respondent(series: pd.Series) -> pd.Series:
    cleaned = series.astype(str).str.strip().replace({"nan": np.nan, "": np.nan})
    cleaned = cleaned.str.replace(r"\\.0$", "", regex=True)
    return cleaned.fillna("").map(lambda value: value.zfill(3) if value else value)


def canonicalize_title(value: str) -> str:
    if pd.isna(value):
        return ""
    text = str(value).strip()
    if not text:
        return ""
    fixes = {
        "abbott elementary": "Abbot Elementary",
        "abbot elementary": "Abbot Elementary",
        "mad max fury road": "Mad Max",
        "mad max": "Mad Max",
        "the town": "The Town",
    }
    return fixes.get(text.lower(), text)


def build_stage2_metadata(rename_map_df: pd.DataFrame, questions_df: pd.DataFrame) -> pd.DataFrame:
    base = rename_map_df.copy()
    base["target_column"] = base["target_column"].astype(str).str.strip()
    base = base.loc[base["target_column"].ne("")]
    base["question_code"] = base["question_code"].astype(str).str.strip().str.upper()

    questions = questions_df.copy()
    questions["question_code"] = questions["question_code"].astype(str).str.strip().str.upper()

    merged = base.merge(questions, on="question_code", how="left", suffixes=("", "_question"))
    merged["question_type"] = merged["question_type"].fillna("likert").str.lower()
    merged["subscale"] = merged["subscale"].fillna("")
    merged["polarity"] = merged["polarity"].fillna("").str.lower()
    merged["question_text"] = merged["question_text"].fillna(merged["suffix_text"]).fillna("")

    extracted = merged["target_column"].str.extract(r"^(Long|Short)_(.*?)_Survey_(.*)$")
    merged["stimulus_form_token"] = extracted[0].fillna("")
    merged["stimulus_title_raw"] = extracted[1].fillna("")
    merged["stimulus_form"] = merged["stimulus_form_token"].map({"Long": "Long Form", "Short": "Short Form"}).fillna("")
    merged["stimulus_title"] = merged["stimulus_title_raw"].apply(canonicalize_title)
    merged["category"] = merged["topic"].fillna("").str.lower()
    merged["questionnaire"] = "Stage 2 Survey"
    return merged


def parse_feature_column(column: str) -> dict | None:
    match = feature_pattern.match(column)
    if not match:
        return None
    form_token, title_raw, instrument, tail = match.groups()
    parts = tail.split("_")
    metric_base = parts[0]
    stat = "_".join(parts[1:]) if len(parts) > 1 else ""
    metric_label = metric_base if not stat else f"{metric_base}_{stat}"
    question_code = f"{DERIVED_PREFIX}{metric_label}"

    subscale = metric_base.replace("Composite", "") or metric_base
    if metric_base.lower().startswith("familiarity"):
        subscale = "Familiarity"
    elif metric_base.lower() == "nostalgia":
        subscale = "Nostalgia"

    category = "familiarity" if "familiarity" in metric_base.lower() else "enjoyment"
    if instrument == "Screening":
        category = "screening_familiarity"

    return {
        "stimulus_form": "Long Form" if form_token == "Long" else "Short Form",
        "stimulus_title": canonicalize_title(title_raw),
        "questionnaire": f"Stage 2 {instrument}",
        "question_code": question_code.upper(),
        "question_text": f"{instrument} {metric_base.replace('Composite', ' Composite')} {stat}".strip(),
        "question_type": "derived_metric",
        "subscale": subscale.title(),
        "category": category.replace("_", " "),
        "score_stat": stat,
        "metric_base": metric_base,
        "instrument": instrument,
    }


In [24]:
results_dir = project_root / "results"

stage2_wide_df = load_or_read("uv_stage2", results_dir / "uv_stage2.csv")
stage2_wide_df["respondent"] = standardize_respondent(stage2_wide_df["respondent"])
if "group" in stage2_wide_df.columns:
    stage2_wide_df["group"] = stage2_wide_df["group"].astype(str).str.strip().str.upper()

stage2_features_df = load_or_read("survey_features", results_dir / "uv_stage2_features.csv")
stage2_features_df["respondent"] = standardize_respondent(stage2_features_df["respondent"])

open_ended_path = results_dir / "uv_stage2_open_ended.csv"
if "survey_open_ended" in globals():
    stage2_open_ended_df = load_or_read("survey_open_ended", open_ended_path)
elif open_ended_path.exists():
    stage2_open_ended_df = pd.read_csv(open_ended_path)
else:
    stage2_open_ended_df = pd.DataFrame()
if not stage2_open_ended_df.empty:
    stage2_open_ended_df["respondent"] = standardize_respondent(stage2_open_ended_df["respondent"])

stage2_issues_path = results_dir / "uv_stage2_issues.csv"
stage2_issues_df = pd.read_csv(stage2_issues_path) if stage2_issues_path.exists() else pd.DataFrame()
if not stage2_issues_df.empty:
    stage2_issues_df["respondent"] = standardize_respondent(stage2_issues_df["respondent"])

stage1_df = load_or_read("uv_stage1_base", results_dir / "uv_stage1.csv")
stage1_df["respondent"] = standardize_respondent(stage1_df["respondent"])
stage1_df["group"] = stage1_df["group"].astype(str).str.strip().str.upper()
stage1_lookup = stage1_df[["respondent", "group", "Short Form", "Long Form"]].drop_duplicates("respondent", keep="last")

rename_map_df = pd.read_csv(project_root / "data" / "survey_column_rename.csv")
questions_df = pd.read_csv(project_root / "data" / "survey_questions.csv")
metadata_df = build_stage2_metadata(rename_map_df, questions_df)

response_columns = sorted(set(metadata_df["target_column"]).intersection(stage2_wide_df.columns))
missing_columns = sorted(set(metadata_df["target_column"]) - set(response_columns))
if missing_columns:
    sample_missing = ", ".join(missing_columns[:5])
    print(
        f"Metadata columns missing from Stage 2 wide export: {len(missing_columns)}"
        + (f" (e.g., {sample_missing})" if sample_missing else "")
    )

id_candidates = [
    "respondent",
    "group",
    "survey_group",
    "survey_study",
    "survey_gender",
    "survey_age",
    "survey_file",
    "source_file",
    "Short Form",
    "Long Form",
]
id_vars = [col for col in id_candidates if col in stage2_wide_df.columns]

response_long = stage2_wide_df.melt(
    id_vars=id_vars,
    value_vars=response_columns,
    var_name="target_column",
    value_name="response_raw",
)
response_long["respondent"] = standardize_respondent(response_long["respondent"])
response_long["response_clean"] = response_long["response_raw"].apply(
    lambda value: clean_response(value) if pd.notna(value) else value
)

numeric_columns = [col for col in response_columns if col in stage2_features_df.columns]
if numeric_columns:
    numeric_long = stage2_features_df.melt(
        id_vars=["respondent"],
        value_vars=numeric_columns,
        var_name="target_column",
        value_name="response_numeric",
    )
    numeric_long["respondent"] = standardize_respondent(numeric_long["respondent"])
else:
    numeric_long = pd.DataFrame(columns=["respondent", "target_column", "response_numeric"])

long_df = response_long.merge(numeric_long, on=["respondent", "target_column"], how="left")

meta_cols = [
    "target_column",
    "question_code",
    "question_text",
    "question_type",
    "subscale",
    "polarity",
    "stimulus_form",
    "stimulus_title",
    "category",
    "questionnaire",
]
long_df = long_df.merge(metadata_df[meta_cols], on="target_column", how="left")

long_df["stimulus_form"] = long_df["stimulus_form"].replace("", np.nan)
long_df["stimulus_form"] = long_df["stimulus_form"].fillna(
    long_df["target_column"].str.extract(r"^(Long|Short)_")[0].map({"Long": "Long Form", "Short": "Short Form"})
)
long_df["stimulus_title"] = long_df["stimulus_title"].replace("", np.nan)
long_df["stimulus_title"] = long_df["stimulus_title"].fillna(
    long_df["target_column"].str.extract(r"^(?:Long|Short)_(.*?)_Survey_")[0].apply(canonicalize_title)
)

long_df = long_df.merge(stage1_lookup, on="respondent", how="left", suffixes=("", "_stage1"))
for column in ["group", "Short Form", "Long Form"]:
    stage1_col = f"{column}_stage1"
    if stage1_col in long_df.columns:
        long_df[column] = long_df[column].fillna(long_df[stage1_col])
        long_df.drop(columns=[stage1_col], inplace=True)

long_df["group"] = long_df.get("group", "").astype(str).str.strip().str.upper()
long_df["question_code"] = long_df["question_code"].fillna("").str.upper()
missing_code_mask = long_df["question_code"].eq("")
long_df.loc[missing_code_mask, "question_code"] = long_df.loc[missing_code_mask, "target_column"].apply(
    lambda col: f"UNKNOWN_{col}"
)
long_df["question_type"] = long_df["question_type"].fillna("unknown").str.lower()
long_df["subscale"] = long_df["subscale"].fillna("").str.strip()
long_df["category"] = long_df["category"].fillna("").str.replace("_", " ").str.strip()
long_df["polarity"] = long_df["polarity"].fillna("")
long_df["questionnaire"] = long_df["questionnaire"].fillna("Stage 2 Survey")
long_df["source_path"] = long_df.get("survey_file")
long_df["submitted_timestamp"] = pd.NaT
long_df["accuracy"] = ""
long_df["value_kind"] = "response"
long_df["score_stat"] = ""


def infer_score_method(question_type: str) -> str:
    if question_type == "likert":
        return "stage2_likert"
    if question_type in {"familiarity", "binary"}:
        return "stage2_familiarity"
    if question_type == "open ended":
        return "stage2_open_ended"
    return "stage2_response"


long_df["score_method"] = long_df["question_type"].apply(infer_score_method)
long_df["score_value"] = long_df["response_numeric"]
EXPLANATION_MAP = {
    "stage2_likert": "Likert item scored on a 1-5 scale with polarity applied.",
    "stage2_familiarity": "Familiarity item normalized per Stage 2 scoring pipeline.",
    "stage2_open_ended": "Open-ended text captured verbatim.",
    "stage2_response": "Stage 2 response value without additional scoring.",
}
long_df["score_explanation"] = long_df["score_method"].map(EXPLANATION_MAP)

response_column_set = set(response_columns)
derived_columns = [
    col for col in stage2_features_df.columns if col not in response_column_set.union({"respondent"})
]
derived_records = []
if derived_columns:
    derived_long = stage2_features_df.melt(
        id_vars=["respondent"],
        value_vars=derived_columns,
        var_name="feature_name",
        value_name="score_value",
    ).dropna(subset=["score_value"], how="all")
    if not derived_long.empty:
        derived_long["respondent"] = standardize_respondent(derived_long["respondent"])
        parsed_meta = derived_long["feature_name"].apply(parse_feature_column)
        valid_mask = parsed_meta.notna()
        skipped_columns = sorted(set(derived_columns) - set(derived_long.loc[valid_mask, "feature_name"].unique()))
        if skipped_columns:
            print(
                f"Skipped {len(skipped_columns)} Stage 2 feature column(s) that do not map to the long-form schema."
            )
        if valid_mask.any():
            derived_augmented = pd.concat(
                [
                    derived_long.loc[valid_mask].reset_index(drop=True),
                    pd.DataFrame(parsed_meta[valid_mask].tolist()),
                ],
                axis=1,
            )
            derived_augmented = derived_augmented.merge(
                stage1_lookup, on="respondent", how="left", suffixes=("", "_stage1")
            )
            for column in ["group", "Short Form", "Long Form"]:
                stage1_col = f"{column}_stage1"
                if stage1_col in derived_augmented.columns:
                    derived_augmented[column] = derived_augmented[column].fillna(derived_augmented[stage1_col])
                    derived_augmented.drop(columns=[stage1_col], inplace=True)
            derived_augmented["group"] = derived_augmented.get("group", "").astype(str).str.strip().str.upper()
            derived_augmented["stimulus_title"] = derived_augmented["stimulus_title"].apply(canonicalize_title)
            derived_augmented["question_text"] = derived_augmented["question_text"].str.strip()
            derived_augmented["question_type"] = derived_augmented["question_type"].str.lower()
            derived_augmented["subscale"] = derived_augmented["subscale"].str.strip()
            derived_augmented["category"] = derived_augmented["category"].str.strip()
            derived_augmented["score_method"] = derived_augmented.apply(
                lambda row: "stage2_feature_"
                + row["instrument"].lower()
                + (f"_{row['score_stat'].lower()}" if row["score_stat"] else ""),
                axis=1,
            )
            derived_augmented["score_explanation"] = derived_augmented.apply(
                lambda row: f"{row['instrument']} {row['metric_base']} metric"
                + (f" ({row['score_stat']})" if row['score_stat'] else ""),
                axis=1,
            )
            derived_augmented["response_raw"] = np.nan
            derived_augmented["response_clean"] = np.nan
            derived_augmented["response_numeric"] = derived_augmented["score_value"]
            derived_augmented["value_kind"] = "score"
            derived_augmented["submitted_timestamp"] = pd.NaT
            derived_augmented["accuracy"] = ""
            derived_augmented["source_path"] = np.nan
            derived_augmented["survey_group"] = np.nan
            derived_augmented["survey_study"] = np.nan
            derived_augmented["survey_gender"] = np.nan
            derived_augmented["survey_age"] = np.nan
            derived_augmented["target_column"] = derived_augmented["feature_name"]
            derived_records.append(
                derived_augmented.drop(columns=["feature_name", "metric_base", "instrument"])
            )

if derived_records:
    derived_df = pd.concat(derived_records, ignore_index=True)
else:
    derived_df = pd.DataFrame(columns=long_df.columns)

stage2_long = pd.concat([long_df, derived_df], ignore_index=True, sort=False)

if not stage2_issues_df.empty:
    issue_map = (
        stage2_issues_df.groupby("respondent")["issue"].apply(
            lambda values: "; ".join(sorted({str(v).strip() for v in values if pd.notna(v)}))
        )
        .reset_index()
        .rename(columns={"issue": "issue_code"})
    )
    stage2_long = stage2_long.merge(issue_map, on="respondent", how="left")
else:
    stage2_long["issue_code"] = np.nan
stage2_long["data_quality_flag"] = stage2_long["issue_code"].notna().map({True: "true", False: "false"})

stage2_long["stimulus_title"] = stage2_long["stimulus_title"].apply(canonicalize_title)
stage2_long["respondent"] = standardize_respondent(stage2_long["respondent"])

column_order = [
    "submitted_timestamp",
    "respondent",
    "group",
    "survey_group",
    "survey_study",
    "survey_gender",
    "survey_age",
    "stimulus_form",
    "stimulus_title",
    "Short Form",
    "Long Form",
    "questionnaire",
    "source_path",
    "question_code",
    "target_column",
    "question_text",
    "question_type",
    "subscale",
    "category",
    "accuracy",
    "response_raw",
    "response_clean",
    "response_numeric",
    "score_value",
    "score_method",
    "score_stat",
    "score_explanation",
    "value_kind",
    "data_quality_flag",
    "issue_code",
]
for column in column_order:
    if column not in stage2_long.columns:
        stage2_long[column] = np.nan
stage2_long = stage2_long.loc[:, column_order]

response_dupes = stage2_long.loc[
    (stage2_long["value_kind"] == "response")
    & stage2_long.duplicated(subset=["respondent", "target_column"], keep=False)
]
missing_fallback = stage2_long["question_code"].str.startswith("UNKNOWN_").sum()

long_output_path = results_dir / "stage2_enjoyment_responses_long.csv"
preview_path = (
    project_root
    / "tasks"
    / "13-11-25 Wrap up enjoyment data"
    / "stage2_enjoyment_responses_long.csv"
)
preview_path.parent.mkdir(parents=True, exist_ok=True)

long_output_path = safe_write_csv(stage2_long, long_output_path)
preview_path = safe_write_csv(stage2_long, preview_path)

print(
    f"Stage 2 long-form export contains {stage2_long.shape[0]:,} rows across "
    f"{stage2_long['respondent'].nunique()} respondents and "
    f"{stage2_long['question_code'].nunique()} unique question codes."
)
if not response_dupes.empty:
    print(
        f"Warning: {response_dupes.shape[0]} response row(s) share the same respondent/target_column combination."
    )
if missing_fallback:
    print(f"Warning: {missing_fallback} row(s) used fallback question codes.")
if not stage2_issues_df.empty:
    print(
        f"Stage 2 issues propagated for {stage2_issues_df['respondent'].nunique()} respondent(s)."
    )

stage2_long.head(10)


Metadata columns missing from Stage 2 wide export: 23 (e.g., General_Abbot Elementary (Abbot Elementary vs Mad Max)_Survey_Familiarity_F3, General_Abbot Elementary (Abbot Elementary vs Mad Max)_Survey_Familiarity_F4, General_Abbot Elementary (The Town vs Abbot Elementary)_Survey_Familiarity_F3, General_Abbot Elementary (The Town vs Abbot Elementary)_Survey_Familiarity_F4, General_Abbot Elementary vs Mad Max_Survey_Familiarity_F5)
Skipped 8 Stage 2 feature column(s) that do not map to the long-form schema.
Stage 2 long-form export contains 18,367 rows across 83 respondents and 69 unique question codes.
Stage 2 issues propagated for 1 respondent(s).


,submitted_timestamp,respondent,group,survey_group,survey_study,survey_gender,survey_age,stimulus_form,stimulus_title,Short Form,...,response_raw,response_clean,response_numeric,score_value,score_method,score_stat,score_explanation,value_kind,data_quality_flag,issue_code
0,NaT,104,A,Default,Group A,MALE,59.0,Long Form,Abbot Elementary,Mad Max,...,NaN,NaN,NaN,NaN,stage2_likert,,Likert item scored on a 1-5 scale with polarit...,response,false,NaN
1,NaT,106,A,Default,Group A,MALE,30.0,Long Form,Abbot Elementary,Mad Max,...,NaN,NaN,NaN,NaN,stage2_likert,,Likert item scored on a 1-5 scale with polarit...,response,false,NaN
2,NaT,116,A,Default,Group A,MALE,19.0,Long Form,Abbot Elementary,Mad Max,...,NaN,NaN,NaN,NaN,stage2_likert,,Likert item scored on a 1-5 scale with polarit...,response,false,NaN
3,NaT,014,A,Default,Group A,MALE,33.0,Long Form,Abbot Elementary,Mad Max,...,NaN,NaN,NaN,NaN,stage2_likert,,Likert item scored on a 1-5 scale with polarit...,response,false,NaN
4,NaT,003,A,Default,Group A,FEMALE,34.0,Long Form,Abbot Elementary,Mad Max,...,NaN,NaN,NaN,NaN,stage2_likert,,Likert item scored on a 1-5 scale with polarit...,response,false,NaN
5,NaT,052,A,Default,Group A,FEMALE,50.0,Long Form,Abbot Elementary,Mad Max,...,NaN,NaN,NaN,NaN,stage2_likert,,Likert item scored on a 1-5 scale with polarit...,response,false,NaN
6,NaT,055,A,Default,Group A,FEMALE,37.0,Long Form,Abbot Elementary,Mad Max,...,NaN,NaN,NaN,NaN,stage2_likert,,Likert item scored on a 1-5 scale with polarit...,response,false,NaN
7,NaT,008,A,Default,Group A,OTHER,51.0,Long Form,Abbot Elementary,Mad Max,...,NaN,NaN,NaN,NaN,stage2_likert,,Likert item scored on a 1-5 scale with polarit...,response,false,NaN
8,NaT,081,A,Default,Group A,FEMALE,24.0,Long Form,Abbot Elementary,Mad Max,...,NaN,NaN,NaN,NaN,stage2_likert,,Likert item scored on a 1-5 scale with polarit...,response,false,NaN
9,NaT,083,A,Default,Group A,MALE,69.0,Long Form,Abbot Elementary,Mad Max,...,NaN,NaN,NaN,NaN,stage2_likert,,Likert item scored on a 1-5 scale with polarit...,response,false,NaN


In [25]:
import json

# Stage 2 validation summary to confirm export integrity.
stage2_long_responses = stage2_long[stage2_long["value_kind"] == "response"].copy()
stage2_long_scores = stage2_long[stage2_long["value_kind"] == "score"].copy()

expected_response_rows = len(stage2_wide_df) * len(response_columns)
actual_response_rows = stage2_long_responses.shape[0]
non_null_responses = stage2_long_responses["response_clean"].notna().sum()
empty_responses = actual_response_rows - non_null_responses
duplicate_response_pairs = stage2_long_responses.duplicated(["respondent", "target_column"], keep=False).sum()

respondent_sets_match = set(stage2_long_responses["respondent"].unique()) == set(stage2_wide_df["respondent"].unique())
target_column_coverage = set(stage2_long_responses["target_column"].unique()) == set(response_columns)

validation_summary = {
    "respondents_in_long": int(stage2_long_responses["respondent"].nunique()),
    "respondents_in_wide": int(stage2_wide_df["respondent"].nunique()),
    "respondent_sets_match": respondent_sets_match,
    "response_rows": int(actual_response_rows),
    "expected_response_slots": int(expected_response_rows),
    "duplicate_response_pairs": int(duplicate_response_pairs),
    "non_null_responses": int(non_null_responses),
    "empty_responses": int(empty_responses),
    "score_rows": int(stage2_long_scores.shape[0]),
    "target_column_coverage": target_column_coverage,
    "issues_flagged": int(stage2_long["data_quality_flag"].eq("true").sum()),
}

if duplicate_response_pairs > 0:
    print(f"Warning: Found {duplicate_response_pairs} duplicate response pair(s).")

validation_frame = pd.DataFrame([validation_summary]).T
validation_frame.columns = ["value"]
display(validation_frame)

validation_summary_path = (
    project_root
    / "tasks"
    / "13-11-25 Wrap up enjoyment data"
    / "stage2_validation_summary.json"
)
validation_summary_path.write_text(json.dumps(validation_summary, indent=2))

print(f"Validation summary written to {validation_summary_path.relative_to(project_root)}")

,value
respondents_in_long,83
respondents_in_wide,83
respondent_sets_match,True
response_rows,7553
expected_response_slots,7553
duplicate_response_pairs,0
non_null_responses,2478
empty_responses,5075
score_rows,10814
target_column_coverage,True


Validation summary written to tasks\13-11-25 Wrap up enjoyment data\stage2_validation_summary.json


In [26]:
print(survey_features.loc[(survey_features["respondent"].astype(int)==82)]['Long_Abbot Elementary_Survey_EnjoymentComposite_Sum'])
print(survey_features.loc[(survey_features["respondent"].astype(int)==82)]['Long_Abbot Elementary_Survey_Enjoyment_E18'])
print(survey_features.loc[(survey_features["respondent"].astype(int)==82)]['Long_Abbot Elementary_Survey_EnjoymentComposite_Corrected'])

31    25.0
Name: Long_Abbot Elementary_Survey_EnjoymentComposite_Sum, dtype: float64
31    2.0
Name: Long_Abbot Elementary_Survey_Enjoyment_E18, dtype: float64
31    23.0
Name: Long_Abbot Elementary_Survey_EnjoymentComposite_Corrected, dtype: float64


In [27]:
prefix = "Long_Abbot Elementary"
columns_to_show = [
    f"{prefix}_Survey_EnjoymentComposite_Sum",
    f"{prefix}_Survey_EnjoymentComposite_Corrected",
    f"{prefix}_Survey_EnjoymentComposite_Count",
    f"{prefix}_Survey_EnjoymentComposite_Normalized",
    f"{prefix}_Survey_EnjoymentComposite_NormalizedCorrected",
    f"{prefix}_Survey_EnjoymentComposite_Mean",
]
comparison = survey_features.loc[
    survey_features[f"{prefix}_Survey_EnjoymentComposite_Count"].gt(0),
    columns_to_show
].head(10).copy()
comparison["normalized_from_corrected"] = np.clip(
    (comparison[f"{prefix}_Survey_EnjoymentComposite_Corrected"]
     - comparison[f"{prefix}_Survey_EnjoymentComposite_Count"])
    / (4.0 * comparison[f"{prefix}_Survey_EnjoymentComposite_Count"]),
    0,
    1,
 )
comparison

,Long_Abbot Elementary_Survey_EnjoymentComposite_Sum,Long_Abbot Elementary_Survey_EnjoymentComposite_Corrected,Long_Abbot Elementary_Survey_EnjoymentComposite_Count,Long_Abbot Elementary_Survey_EnjoymentComposite_Normalized,Long_Abbot Elementary_Survey_EnjoymentComposite_NormalizedCorrected,Long_Abbot Elementary_Survey_EnjoymentComposite_Mean,normalized_from_corrected
22,41.0,41.0,12,0.604167,0.604167,3.416667,0.604167
23,51.0,53.0,12,0.812500,0.854167,4.416667,0.854167
24,50.0,54.0,12,0.791667,0.875000,4.500000,0.875000
25,45.0,49.0,12,0.687500,0.770833,4.083333,0.770833
26,51.0,55.0,12,0.812500,0.895833,4.583333,0.895833
27,35.0,37.0,12,0.479167,0.520833,3.083333,0.520833
28,19.0,15.0,12,0.145833,0.062500,1.250000,0.062500
29,19.0,15.0,12,0.145833,0.062500,1.250000,0.062500
30,55.0,55.0,12,0.895833,0.895833,4.583333,0.895833
31,25.0,23.0,12,0.270833,0.229167,1.916667,0.229167


## Stage 3: Post Questionnaire
The seven-day follow-up instrument captures delayed recall, comprehension, and broader post-viewing perceptions. This section documents how the Post questionnaire exports extend the unified view (`uv`) with respondent-level memory, confidence, and engagement measures.

### Planned Workflow
- Inventory every group-level Post questionnaire export under `data/Recall/`, ensuring respondent IDs stay string-typed for clean joins.
- Harmonize column names via `data/post_survey_map.csv` so variants or duplicate headers collapse to a single `question_code`.
- Engineer response features (e.g., accuracy, confidence, free-text summaries) while following the `{form}_{title}_Post_{metric}_{method}` naming pattern.
- Merge the Post feature frame onto the Stage 1 base (preserving respondent metadata) and log gaps to a dedicated issues table.
- Validate the enriched unified view and emit Stage 3 outputs (`uv_stage3.csv`, `uv_stage3_issues.csv`) to `results/` with timestamped fallbacks when needed.


In [31]:
from pathlib import Path
import pandas as pd
import re

post_map_path = project_root / "data" / "post_survey_map.csv"
post_map_df = pd.read_csv(post_map_path)

code_pattern = re.compile(r"^\s*([0-9]+(?:\.[0-9]+)*)")

def extract_question_code(text: str):
    if pd.isna(text):
        return None
    match = code_pattern.match(str(text))
    return match.group(1) if match else None

if "question_code" not in post_map_df.columns:
    post_map_df.insert(1, "question_code", post_map_df["question"].apply(extract_question_code))
else:
    post_map_df["question_code"] = post_map_df["question"].apply(extract_question_code)

post_map_df.to_csv(post_map_path, index=False)
missing_codes = post_map_df["question_code"].isna().sum()
print(
    f"Updated {post_map_path.name} with question_code column; "
    f"{missing_codes} rows are missing a parsed code."
)
post_map_df.head(10)

Updated post_survey_map.csv with question_code column; 2 rows are missing a parsed code.


,question,question_code,type,subscale,category,accuracy,Group A,Group B,Group C,Group D,Group E,Group F
0,1.1. Did you see this scene in the videos you ...,1.1,binary,recognition,key,hit,Mad Max,The Town,The Town,Abbot Elementary,Mad Max,Mad Max
1,2.1. Did you see this scene in the videos you ...,2.1,binary,recognition,fake,miss,The Dark Knight,The Dark Knight,Goodfellas,Goodfellas,Wonder Woman,Wonder Woman
2,3.1. Did you see this scene in the videos you ...,3.1,binary,recognition,distractor,hit,Titanic,The Notebook,The Notebook,Schitts Creek,Schitts Creek,Titanic
3,4.1. Did you see this scene in the videos you ...,4.1,binary,recognition,key,hit,The Town,Mad Max,Abbot Elementary,The Town,Abbot Elementary,Abbot Elementary
4,5.1. Did you see this scene in the videos you ...,5.1,binary,recognition,unseen,miss,Mad Max,The Town,The Town,Abbot Elementary,Abbot Elementary,Mad Max
5,6.1. Did you see this scene in the videos you ...,6.1,binary,recognition,fake,miss,I am Legend,Titanic,Seinfeld,Seinfeld,Friends,Friends
6,7.1. Did you see this scene in the videos you ...,7.1,binary,recognition,seen,hit,The Town,Mad Max,Abbot Elementary,The Town,Mad Max,Abbot Elementary
7,8.1. Did you see this scene in the videos you ...,8.1,binary,recognition,fake,miss,Friends,Friends,Friends,Friends,Seinfeld,Seinfeld
8,9.1. Did you see this scene in the videos you ...,9.1,binary,recognition,seen,hit,The Town,The Town,Abbot Elementary,Abbot Elementary,Abbot Elementary,Mad Max
9,10.1. Did you see this scene in the videos you...,10.1,binary,recognition,distractor,hit,A Star Is Born,Ironman,Ironman,The Office,The Office,A Star Is Born


In [32]:
import numpy as np
import re

post_data_dir = project_root / "data" / "Post"
if not post_data_dir.exists():
    raise FileNotFoundError(f"Post questionnaire directory not found at {post_data_dir}.")
post_files = sorted(
    path
    for path in post_data_dir.rglob("*.csv")
    if path.is_file() and not path.name.startswith("~$")
)
if not post_files:
    raise FileNotFoundError(f"No post questionnaire CSV files found under {post_data_dir}.")

def merge_duplicate_columns(df: pd.DataFrame) -> pd.DataFrame:
    def _sanitize(value):
        if pd.isna(value):
            return np.nan
        if isinstance(value, str):
            cleaned = value.strip()
            return cleaned if cleaned else np.nan
        return value
    merged = {}
    column_order = []
    for column in df.columns:
        normalized = re.sub(r"\s+", " ", str(column)).strip()
        series = df[column].astype("object").map(_sanitize)
        if normalized not in merged:
            merged[normalized] = series
            column_order.append(normalized)
        else:
            merged[normalized] = merged[normalized].combine_first(series)
    return pd.DataFrame({name: merged[name] for name in column_order})

post_map_full = pd.read_csv(project_root / "data" / "post_survey_map.csv")
if "question_code" not in post_map_full.columns:
    post_map_full.insert(1, "question_code", post_map_full["question"].apply(extract_question_code))
else:
    post_map_full["question_code"] = post_map_full["question"].apply(extract_question_code)
post_map_full["subscale"] = post_map_full["subscale"].astype(str)

CATEGORY_RENAME = {
    "key": "wb-key",
    "seen": "wb-notKeySeen",
    "unseen": "wb-notKeyUnseen",
    "fake": "distractor",
    "distractor": "comp-key",
    "distractor2": "comp-notKeySeen",
}

STAT_LABELS = {
    "count": "Count",
    "sum": "Sum",
    "mean": "Mean",
    "normalized_mean": "NormalizedMean",
}
RECOGNITION_BINARY_MAX = 2.0
RECOGNITION_CONFIDENCE_MAX = 4.0
RECOGNITION_COMPOSITE_MAX = RECOGNITION_BINARY_MAX * RECOGNITION_CONFIDENCE_MAX
FORM_CATEGORY_KEYS = {"key", "seen"}
NON_FORM_CATEGORY_KEYS = {"unseen", "fake", "distractor", "distractor2"}

uv_stage1_path = project_root / "results" / "uv_stage1.csv"
if not uv_stage1_path.exists():
    raise FileNotFoundError(f"Stage 1 dataset not found at {uv_stage1_path}.")
uv_stage1 = pd.read_csv(uv_stage1_path)
uv_stage1["respondent"] = uv_stage1["respondent"].astype(str).str.strip()
uv_stage1["group"] = uv_stage1["group"].astype(str).str.strip().str.upper()

uv_stage2_path = project_root / "results" / "uv_stage2.csv"
uv_stage2_cached = None
if uv_stage2_path.exists():
    uv_stage2_cached = pd.read_csv(uv_stage2_path)
    uv_stage2_cached["respondent"] = uv_stage2_cached["respondent"].astype(str).str.strip()

uv_stage1_lookup_df = (
    uv_stage1
    .loc[:, ["respondent", "group", "Short Form", "Long Form"]]
    .dropna(subset=["respondent"])
    .assign(respondent=lambda df: df["respondent"].astype(str).str.strip())
    .drop_duplicates("respondent", keep="last")
)
uv_stage1_lookup = uv_stage1_lookup_df.set_index("respondent").to_dict("index")

recognition_map = post_map_full.loc[
    post_map_full["subscale"].str.lower() == "recognition",
    :,
]
recognition_map = recognition_map.loc[recognition_map["question_code"].notna()].copy()
recognition_map["question_code"] = recognition_map["question_code"].astype(str).str.strip()
recognition_map = recognition_map.loc[recognition_map["question_code"].str.match(r"^\d+\.[12]$")]

valid_question_codes = set(recognition_map["question_code"].unique())

group_columns = [col for col in recognition_map.columns if col.startswith("Group ")]

TITLE_NORMALIZATION = {
    "abbott elementary": "Abbot Elementary",
    "abbot elementary": "Abbot Elementary",
    "schitts creek": "Schittss Creek",
    "schittss creek": "Schittss Creek",
    "mad max fury road": "Mad Max",
    "mad max": "Mad Max",
}

def canonicalize_title(raw_title: str) -> str:
    if pd.isna(raw_title):
        return ""
    cleaned = str(raw_title).strip()
    if not cleaned:
        return ""
    lookup_key = cleaned.lower()
    return TITLE_NORMALIZATION.get(lookup_key, cleaned)

respondent_exposures: dict[str, dict[str, set[str]]] = {}
for respondent_id, info in uv_stage1_lookup.items():
    exposures: dict[str, set[str]] = {}
    long_title = canonicalize_title(info.get("Long Form", ""))
    if long_title:
        exposures.setdefault("Long", set()).add(long_title)
    short_title = canonicalize_title(info.get("Short Form", ""))
    if short_title:
        exposures.setdefault("Short", set()).add(short_title)
    if exposures:
        respondent_exposures[str(respondent_id)] = exposures

meta_lookup = {}
for _, row in recognition_map.iterrows():
    q_code = row["question_code"]
    q_root, q_suffix = q_code.split(".")
    category = str(row.get("category", "")).strip()
    accuracy = str(row.get("accuracy", "")).strip().lower()
    for group_col in group_columns:
        group_letter = group_col.replace("Group ", "").strip().upper()
        title_value = canonicalize_title(row.get(group_col, ""))
        meta_lookup[(group_letter, q_root, q_suffix)] = {
            "title": title_value,
            "category": category,
            "accuracy": accuracy,
        }

stimulus_map = pd.read_csv(project_root / "data" / "stimulus_rename.csv")
stimulus_map["group_letter"] = stimulus_map["group"].str.extract(r"([A-F])", expand=False)
stimulus_map["title_clean"] = stimulus_map["title"].map(canonicalize_title)
stimulus_map["form_clean"] = stimulus_map["form"].astype(str).str.title()

group_title_form_lookup = {
    (row.group_letter, row.title_clean): row.form_clean
    for row in stimulus_map.itertuples()
    if isinstance(row.group_letter, str) and isinstance(row.title_clean, str) and row.title_clean
}

if uv_stage2_cached is not None:
    uv_columns = pd.Index(uv_stage2_cached.columns)
else:
    uv_columns = pd.Index(uv_stage1.columns)

yes_values = {"yes", "y", "true", "1"}
no_values = {"no", "n", "false", "0"}

confidence_pattern = re.compile(r"^\s*([0-9]+(?:\.[0-9]+)?)")

def extract_scalar(value):
    if isinstance(value, pd.Series):
        non_null = value.dropna()
        if non_null.empty:
            return np.nan
        return non_null.iloc[0]
    return value

def parse_yes_no(value) -> float:
    value = extract_scalar(value)
    if pd.isna(value):
        return np.nan
    text = str(value).strip().lower()
    if not text:
        return np.nan
    if text in yes_values:
        return 1.0
    if text in no_values:
        return 0.0
    return np.nan

def parse_confidence(value) -> float:
    value = extract_scalar(value)
    if pd.isna(value):
        return np.nan
    text = str(value).strip()
    if not text:
        return np.nan
    match = confidence_pattern.match(text)
    if match:
        try:
            return float(match.group(1))
        except ValueError:
            return np.nan
    try:
        return float(text)
    except ValueError:
        return np.nan

def resolve_form(respondent_id: str, group_letter: str, title: str) -> tuple[str, list[str]]:
    notes: list[str] = []
    exposures = uv_stage1_lookup.get(respondent_id, {})
    long_title = canonicalize_title(exposures.get("Long Form", "")) if exposures else ""
    short_title = canonicalize_title(exposures.get("Short Form", "")) if exposures else ""
    canonical = canonicalize_title(title)
    if canonical and canonical == long_title:
        return "Long", notes
    if canonical and canonical == short_title:
        return "Short", notes
    group_clean = (group_letter or "").strip().upper()
    if canonical:
        mapped = group_title_form_lookup.get((group_clean, canonical))
        if mapped:
            notes.append("form_from_group_stimulus_map")
            return mapped, notes
        has_short = any(col.startswith(f"Short_{canonical}") for col in uv_columns)
        has_long = any(col.startswith(f"Long_{canonical}") for col in uv_columns)
        if has_short and not has_long:
            notes.append("form_inferred_short_from_uv")
            return "Short", notes
        if has_long and not has_short:
            notes.append("form_inferred_long_from_uv")
            return "Long", notes
        if has_short and has_long:
            notes.append("form_ambiguous_default_short")
            return "Short", notes
    notes.append("form_unresolved_default_short")
    return "Short", notes

recognition_records: list[dict] = []
issue_records: list[dict] = []
respondent_post_paths: dict[str, str] = {}
excluded_group_mismatch = set()

for csv_path in post_files:
    file_group_match = re.search(r"Group\s+([A-F])", csv_path.stem, re.IGNORECASE)
    fallback_group_letter = file_group_match.group(1).upper() if file_group_match else ""
    df_raw = pd.read_csv(csv_path, dtype=str)
    df_merged = merge_duplicate_columns(df_raw)
    rename_map = {}
    for column in df_merged.columns:
        q_code = extract_question_code(column)
        if q_code and q_code in valid_question_codes:
            rename_map[column] = q_code
    df_standard = df_merged.rename(columns=rename_map)
    respondent_col = next(((
        col for col in df_standard.columns
        if "participant number" in col.lower()
    )), None)
    if respondent_col is None:
        raise KeyError(f"Participant identifier column not found in {csv_path.name}")
    df_standard["respondent"] = df_standard[respondent_col].astype(str).str.strip()
    df_standard["respondent"] = df_standard["respondent"].replace({"": np.nan, "nan": np.nan})
    df_standard["respondent"] = df_standard["respondent"].str.replace(r"\.0$", "", regex=True)
    if "Timestamp" in df_standard.columns:
        df_standard["timestamp_iso"] = pd.to_datetime(df_standard["Timestamp"], errors="coerce")
    else:
        df_standard["timestamp_iso"] = pd.NaT

    for _, row in df_standard.iterrows():
        respondent_id = row.get("respondent")
        if pd.isna(respondent_id):
            continue
        respondent_id = str(respondent_id).strip()
        if not respondent_id:
            continue
        respondent_info = uv_stage1_lookup.get(respondent_id, {})
        stage1_group = str(respondent_info.get("group", "")).strip().upper()
        post_group_letter = fallback_group_letter
        if stage1_group and post_group_letter and stage1_group != post_group_letter:
            if respondent_id not in excluded_group_mismatch:
                issue_records.append({
                    "respondent": respondent_id,
                    "group": stage1_group,
                    "question_number": np.nan,
                    "title": "",
                    "issue": f"post_group_mismatch_uv_{stage1_group}_file_{post_group_letter}",
                    "source_file": csv_path.name,
                })
            excluded_group_mismatch.add(respondent_id)
            continue
        group_letter = stage1_group or post_group_letter
        if not group_letter:
            issue_records.append({
                "respondent": respondent_id,
                "group": "",
                "question_number": np.nan,
                "title": "",
                "issue": "group_missing_in_stage1_and_filename",
                "source_file": csv_path.name,
            })
            continue
        if respondent_id not in respondent_post_paths:
            try:
                relative_path = csv_path.relative_to(project_root)
                respondent_post_paths[respondent_id] = relative_path.as_posix()
            except ValueError:
                respondent_post_paths[respondent_id] = csv_path.as_posix()
        for q_num in map(str, range(1, 13)):
            base_meta = meta_lookup.get((group_letter, q_num, "1"))
            conf_meta = meta_lookup.get((group_letter, q_num, "2"))
            if base_meta is None or conf_meta is None:
                continue
            binary_value = row.get(f"{q_num}.1")
            confidence_value = row.get(f"{q_num}.2")
            yes_no = parse_yes_no(binary_value)
            confidence = parse_confidence(confidence_value)
            issues_here: list[str] = []
            if np.isnan(yes_no):
                issues_here.append("binary_response_missing_or_unrecognized")
            accuracy = base_meta.get("accuracy", "")
            expected_yes = accuracy == "hit"
            base_score_raw = np.nan
            base_score = np.nan
            if not np.isnan(yes_no):
                answered_yes = bool(yes_no)
                base_score_raw = RECOGNITION_BINARY_MAX if answered_yes == expected_yes else 0.0
                base_score = base_score_raw / RECOGNITION_BINARY_MAX
            if np.isnan(confidence):
                issues_here.append("confidence_missing_or_unrecognized")
            composite_raw = np.nan
            composite = np.nan
            if not np.isnan(base_score_raw) and not np.isnan(confidence):
                composite_raw = base_score_raw * confidence
                composite = composite_raw / RECOGNITION_COMPOSITE_MAX
            form_value, form_notes = resolve_form(respondent_id, group_letter, base_meta.get("title", ""))
            issues_here.extend(form_notes)
            question_int = int(float(q_num))
            title_segment = base_meta.get("title", "").strip() or f"Question {question_int}"
            composite_name = f"{form_value}_{title_segment}_Post_RecognitionComposite_Q{question_int:02d}"
            base_name = f"{form_value}_{title_segment}_Post_Recognition_Q{question_int}-1"
            confidence_name = f"{form_value}_{title_segment}_Post_Recognition_Q{question_int}-2"
            records_to_add = [
                ("composite", composite_name, composite),
                ("binary", base_name, base_score),
                ("confidence", confidence_name, confidence),
            ]
            for metric_label, feature_name, feature_value in records_to_add:
                recognition_records.append({
                    "respondent": respondent_id,
                    "group": group_letter,
                    "question_number": question_int,
                    "title": base_meta.get("title", ""),
                    "category": base_meta.get("category", ""),
                    "accuracy": accuracy,
                    "form": form_value,
                    "metric": metric_label,
                    "column_name": feature_name,
                    "value": feature_value,
                    "timestamp": row.get("timestamp_iso", pd.NaT),
                })
            for issue in issues_here:
                issue_records.append({
                    "respondent": respondent_id,
                    "group": group_letter,
                    "question_number": question_int,
                    "title": base_meta.get("title", ""),
                    "issue": issue,
                    "source_file": csv_path.name,
                })

recognition_df = pd.DataFrame(recognition_records)
if recognition_df.empty:
    raise ValueError("No recognition responses were parsed from post questionnaire files.")
recognition_df = recognition_df.sort_values(["respondent", "column_name", "timestamp"])
recognition_df = recognition_df.drop_duplicates(["respondent", "column_name"], keep="last")

recognition_features = (
    recognition_df
    .pivot(index="respondent", columns="column_name", values="value")
    .sort_index(axis=1)
    .reset_index()
)
recognition_features.columns.name = None

composite_subset = recognition_df.loc[recognition_df["metric"] == "composite"].copy()
if not composite_subset.empty:
    composite_subset["category_lower"] = (
        composite_subset["category"]
        .astype(str)
        .str.strip()
        .str.lower()
        .replace("", np.nan)
    )
    composite_subset = composite_subset.loc[composite_subset["category_lower"].notna()]
    composite_subset["title_clean"] = composite_subset["title"].map(canonicalize_title)
    composite_subset["respondent"] = composite_subset["respondent"].astype(str).str.strip()

    form_subset = composite_subset.loc[composite_subset["category_lower"].isin(FORM_CATEGORY_KEYS)].copy()
    if not form_subset.empty:
        def _match_form(row) -> str | None:
            exposures = respondent_exposures.get(row["respondent"], {})
            for form_label, titles in exposures.items():
                if row["title_clean"] in titles:
                    return form_label
            return None

        form_subset["aligned_form"] = form_subset.apply(_match_form, axis=1)
        unmatched_mask = form_subset["aligned_form"].isna()
        if unmatched_mask.any():
            for row in form_subset.loc[unmatched_mask, ["respondent", "question_number", "title", "category_lower"]].itertuples(index=False):
                respondent_key = str(row.respondent)
                issue_records.append({
                    "respondent": respondent_key,
                    "group": uv_stage1_lookup.get(respondent_key, {}).get("group", ""),
                    "question_number": int(row.question_number) if not pd.isna(row.question_number) else np.nan,
                    "title": row.title,
                    "issue": "post_form_title_not_in_stage1_exposures",
                    "source_file": respondent_post_paths.get(respondent_key, ""),
                })
            form_subset = form_subset.loc[~unmatched_mask]
        if not form_subset.empty:
            form_agg = (
                form_subset
                .groupby(["respondent", "aligned_form", "category_lower"])["value"]
                .agg(count="count", sum="sum", mean="mean")
                .reset_index()
            )
            if not form_agg.empty:
                form_agg["normalized_mean"] = form_agg["mean"].astype(float).clip(0, 1)
                form_long = form_agg.melt(
                    id_vars=["respondent", "aligned_form", "category_lower"],
                    value_vars=["count", "sum", "mean", "normalized_mean"],
                    var_name="statistic",
                    value_name="stat_value",
                )
                form_long["column_name"] = form_long.apply(
                    lambda r: f"{r['aligned_form']}_{r['category_lower']}_Post_Recognition_{STAT_LABELS[r['statistic']]}"
                    , axis=1,
                )
                form_wide = form_long.pivot(index="respondent", columns="column_name", values="stat_value").reset_index()
                form_wide.columns.name = None
                recognition_features = recognition_features.merge(form_wide, on="respondent", how="left")

    non_form_subset = composite_subset.loc[composite_subset["category_lower"].isin(NON_FORM_CATEGORY_KEYS)].copy()
    if not non_form_subset.empty:
        non_form_agg = (
            non_form_subset
            .groupby(["respondent", "category_lower"])["value"]
            .agg(count="count", sum="sum", mean="mean")
            .reset_index()
        )
        if not non_form_agg.empty:
            non_form_agg["normalized_mean"] = non_form_agg["mean"].astype(float).clip(0, 1)
            non_form_long = non_form_agg.melt(
                id_vars=["respondent", "category_lower"],
                value_vars=["count", "sum", "mean", "normalized_mean"],
                var_name="statistic",
                value_name="stat_value",
            )
            non_form_long["column_name"] = non_form_long.apply(
                lambda r: f"{r['category_lower']}_Post_Recognition_{STAT_LABELS[r['statistic']]}"
                , axis=1,
            )
            non_form_wide = non_form_long.pivot(index="respondent", columns="column_name", values="stat_value").reset_index()
            non_form_wide.columns.name = None
            recognition_features = recognition_features.merge(non_form_wide, on="respondent", how="left")

if respondent_post_paths:
    post_path_df = (
        pd.Series(respondent_post_paths, name="post_survey_source_path")
        .to_frame()
        .reset_index()
        .rename(columns={"index": "respondent"})
    )
    recognition_features = recognition_features.merge(post_path_df, on="respondent", how="left")

feature_columns = [col for col in recognition_features.columns if col != "respondent"]
composite_columns = [col for col in feature_columns if "_Post_RecognitionComposite_" in col]
raw_columns = [
    col
    for col in feature_columns
    if "_Post_Recognition_" in col and "Composite" not in col and not col.endswith("source_path")
]

issues_df = pd.DataFrame(issue_records)
if not issues_df.empty:
    issues_df = issues_df.sort_values(["respondent", "question_number", "issue"])

if excluded_group_mismatch:
    def _mismatch_sort_key(value: str):
        text = str(value)
        if text.isdigit():
            return (0, int(text))
        return (1, text)
    excluded_display = ", ".join(sorted(excluded_group_mismatch, key=_mismatch_sort_key))
    print(
        f"Skipped {len(excluded_group_mismatch)} respondent(s) due to post group mismatch: {excluded_display}."
    )

print(
    f"Parsed {recognition_df.shape[0]} recognition feature rows across "
    f"{len(composite_columns)} composite columns and {len(raw_columns)} raw response columns "
    f"for {recognition_features.shape[0]} respondents.",
)
sample_columns = composite_columns[:3] + raw_columns[:3]
if sample_columns:
    print("Sample feature columns:")
    for name in sample_columns:
        print(f"  {name}")
recognition_features.head()


Skipped 3 respondent(s) due to post group mismatch: 6, 116, 117.
Parsed 2880 recognition feature rows across 39 composite columns and 110 raw response columns for 80 respondents.
Sample feature columns:
  Long_Abbot Elementary_Post_RecognitionComposite_Q04
  Long_Abbot Elementary_Post_RecognitionComposite_Q07
  Long_Abbot Elementary_Post_RecognitionComposite_Q09
  Long_Abbot Elementary_Post_Recognition_Q4-1
  Long_Abbot Elementary_Post_Recognition_Q4-2
  Long_Abbot Elementary_Post_Recognition_Q7-1


,respondent,Long_Abbot Elementary_Post_RecognitionComposite_Q04,Long_Abbot Elementary_Post_RecognitionComposite_Q07,Long_Abbot Elementary_Post_RecognitionComposite_Q09,Long_Abbot Elementary_Post_Recognition_Q4-1,Long_Abbot Elementary_Post_Recognition_Q4-2,Long_Abbot Elementary_Post_Recognition_Q7-1,Long_Abbot Elementary_Post_Recognition_Q7-2,Long_Abbot Elementary_Post_Recognition_Q9-1,Long_Abbot Elementary_Post_Recognition_Q9-2,...,distractor_Post_Recognition_Sum,fake_Post_Recognition_Count,fake_Post_Recognition_Mean,fake_Post_Recognition_NormalizedMean,fake_Post_Recognition_Sum,unseen_Post_Recognition_Count,unseen_Post_Recognition_Mean,unseen_Post_Recognition_NormalizedMean,unseen_Post_Recognition_Sum,post_survey_source_path
0,1,1.0,0.75,NaN,1.0,4.0,1.0,3.0,NaN,NaN,...,0.00,2.0,0.3750,0.3750,0.75,1.0,0.00,0.00,0.00,data/Post/Group C_Post Viewing Questionnaire P...
1,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.75,4.0,0.8125,0.8125,3.25,1.0,0.75,0.75,0.75,data/Post/Group B_ Post Viewing Questionnaire ...
2,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.00,4.0,1.0000,1.0000,4.00,1.0,1.00,1.00,1.00,data/Post/Group E_ Post Viewing Questionnaire ...
3,101,1.0,1.00,1.0,1.0,4.0,1.0,4.0,1.0,4.0,...,1.00,4.0,0.6250,0.6250,2.50,1.0,0.50,0.50,0.50,data/Post/Group C_ Post Viewing Questionnaire ...
4,102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.00,4.0,1.0000,1.0000,4.00,1.0,0.00,0.00,0.00,data/Post/Group D_ Post Viewing Questionnaire ...


In [33]:
feature_columns = [col for col in recognition_features.columns if col != "respondent"]
composite_columns = [col for col in feature_columns if "_Post_RecognitionComposite_" in col]
raw_columns = [col for col in feature_columns if "_Post_Recognition_" in col and "Composite" not in col]

print(f"Total recognition feature columns: {len(feature_columns)}")
print(f"Composite columns ({len(composite_columns)} total):")
for name in composite_columns[:20]:
    print(f"  {name}")
if len(composite_columns) > 20:
    print("  ...")
print(f"Raw recognition columns ({len(raw_columns)} total):")
for name in raw_columns[:20]:
    print(f"  {name}")
if len(raw_columns) > 20:
    print("  ...")

Total recognition feature columns: 150
Composite columns (39 total):
  Long_Abbot Elementary_Post_RecognitionComposite_Q04
  Long_Abbot Elementary_Post_RecognitionComposite_Q07
  Long_Abbot Elementary_Post_RecognitionComposite_Q09
  Long_Mad Max_Post_RecognitionComposite_Q01
  Long_Mad Max_Post_RecognitionComposite_Q04
  Long_Mad Max_Post_RecognitionComposite_Q07
  Long_The Town_Post_RecognitionComposite_Q04
  Long_The Town_Post_RecognitionComposite_Q07
  Long_The Town_Post_RecognitionComposite_Q09
  Short_A Star Is Born_Post_RecognitionComposite_Q10
  Short_A Star Is Born_Post_RecognitionComposite_Q11
  Short_A Star Is Born_Post_RecognitionComposite_Q12
  Short_Abbot Elementary_Post_RecognitionComposite_Q01
  Short_Abbot Elementary_Post_RecognitionComposite_Q04
  Short_Abbot Elementary_Post_RecognitionComposite_Q05
  Short_Abbot Elementary_Post_RecognitionComposite_Q09
  Short_Friends_Post_RecognitionComposite_Q06
  Short_Friends_Post_RecognitionComposite_Q08
  Short_Goodfellas_Post_R

In [34]:
stage1_path = project_root / "results" / "uv_stage1.csv"
if not stage1_path.exists():
    raise FileNotFoundError(f"Stage 1 output not found at {stage1_path}.")
uv_stage1_base = pd.read_csv(stage1_path)
uv_stage1_base["respondent"] = uv_stage1_base["respondent"].astype(str).str.strip()

form_columns = [col for col in ("Short Form", "Long Form") if col in uv_stage1_base.columns]
forms_lookup = uv_stage1_base[["respondent", *form_columns]].copy() if form_columns else None
if forms_lookup is not None:
    forms_lookup["respondent"] = forms_lookup["respondent"].astype(str).str.strip()
    if forms_lookup["respondent"].duplicated().any():
        dup_count = forms_lookup["respondent"].duplicated(keep=False).sum()
        print(f"Warning: {dup_count} duplicate form records detected; using the last occurrence per respondent.")
        forms_lookup = forms_lookup.drop_duplicates(subset="respondent", keep="last")

recognition_features = recognition_features.copy()
recognition_features["respondent"] = recognition_features["respondent"].astype(str).str.strip()

uv_stage3 = uv_stage1_base.copy()
if forms_lookup is not None:
    uv_stage3 = uv_stage3.drop(columns=form_columns, errors="ignore")
    uv_stage3 = uv_stage3.merge(forms_lookup, on="respondent", how="left", validate="one_to_one")

uv_stage3 = uv_stage3.merge(recognition_features, on="respondent", how="left")
uv_stage3 = uv_stage3.sort_values("respondent").reset_index(drop=True)

results_dir = project_root / "results"
results_dir.mkdir(parents=True, exist_ok=True)
stage3_path = results_dir / "uv_stage3.csv"
uv_stage3.to_csv(stage3_path, index=False)
print(f"Stage 3 UV shape: {uv_stage3.shape}; saved to {stage3_path.name}.")

issues_path = results_dir / "uv_stage3_issues.csv"
if issues_df.empty:
    if issues_path.exists():
        issues_path.unlink()
    print("No Stage 3 recognition parsing issues detected.")
else:
    issues_df.to_csv(issues_path, index=False)
    print(f"Logged {issues_df.shape[0]} recognition parsing issues to {issues_path.name}.")

uv_stage3


Stage 3 UV shape: (83, 167); saved to uv_stage3.csv.
Logged 599 recognition parsing issues to uv_stage3_issues.csv.


,source_file,group,respondent,date_study,time_study,age,age_group,gender,ethnicity,income_group,...,distractor_Post_Recognition_Sum,fake_Post_Recognition_Count,fake_Post_Recognition_Mean,fake_Post_Recognition_NormalizedMean,fake_Post_Recognition_Sum,unseen_Post_Recognition_Count,unseen_Post_Recognition_Mean,unseen_Post_Recognition_NormalizedMean,unseen_Post_Recognition_Sum,post_survey_source_path
0,008_1.csv,C,1,10/07/2025,10:32:16,63,60-69,Male,Hispanic/Latino/Latina/Latinx,"$60,000 or more per year",...,0.00,2.0,0.3750,0.3750,0.75,1.0,0.00,0.00,0.00,data/Post/Group C_Post Viewing Questionnaire P...
1,005_10.csv,B,10,10/10/2025,13:17:14,65,60-69,Male,White,"$35,000  $60,000 per year",...,1.75,4.0,0.8125,0.8125,3.25,1.0,0.75,0.75,0.75,data/Post/Group B_ Post Viewing Questionnaire ...
2,005_100.csv,E,100,10/15/2025,19:14:06,25,18-27,Female,White,"$35,000  $60,000 per year",...,2.00,4.0,1.0000,1.0000,4.00,1.0,1.00,1.00,1.00,data/Post/Group E_ Post Viewing Questionnaire ...
3,003_101.csv,C,101,10/16/2025,14:58:27,67,60-69,Male,Black/African American,"$60,000 or more per year",...,1.00,4.0,0.6250,0.6250,2.50,1.0,0.50,0.50,0.50,data/Post/Group C_ Post Viewing Questionnaire ...
4,001_102.csv,D,102,10/16/2025,16:35:40,37,28-43,Male,Black/African American,"$35,000  $60,000 per year",...,2.00,4.0,1.0000,1.0000,4.00,1.0,0.00,0.00,0.00,data/Post/Group D_ Post Viewing Questionnaire ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,004_94.csv,D,94,10/15/2025,11:33:19,32,28-43,Male,White,"$35,000  $60,000 per year",...,2.00,4.0,1.0000,1.0000,4.00,1.0,0.00,0.00,0.00,data/Post/Group D_ Post Viewing Questionnaire ...
79,004_96.csv,F,96,10/15/2025,14:32:00,29,28-43,Female,White,"$60,000 or more per year",...,2.00,4.0,1.0000,1.0000,4.00,1.0,1.00,1.00,1.00,data/Post/Group F_ Post Viewing Questionnaire ...
80,004_97.csv,E,97,10/15/2025,17:41:01,32,28-43,Female,White,"$60,000 or more per year",...,2.00,4.0,1.0000,1.0000,4.00,1.0,0.00,0.00,0.00,data/Post/Group E_ Post Viewing Questionnaire ...
81,004_98.csv,A,98,10/15/2025,17:41:49,32,28-43,Male,White,"$35,000  $60,000 per year",...,2.00,4.0,0.8750,0.8750,3.50,1.0,0.50,0.50,0.50,data/Post/Group A_ Post Viewing Questionnaire ...


## Stage 4: Open-Ended Integration
- Merge Stage 2 and Stage 3 open-ended responses onto the Stage 1 baseline to maintain the respondent-level schema.
- Filter prompts via `data/survey_questions.csv` and `data/post_survey_map.csv` so the export captures the open-ended set plus required follow-ups (`E21`, `E22`, `WBD1`, `WBD2`, `Q15`, `Q18`).
- Normalize legacy `question1` markers to `E22` before applying the `{form}_{title}_{Survey/Post}_{subscale}_{question_code}` naming pattern.
- Persist the enriched table to `results/uv_open_ended.csv` for downstream qualitative reviews.

In [35]:
# Stage 4: compile open-ended survey and post responses within the notebook
import csv
import re
import sys
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, Iterable, List, Set, cast

import pandas as pd

project_root = globals().get("project_root", Path.cwd().parent)
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

BASE_DIR = project_root
DATA_DIR = BASE_DIR / "data"
RESULTS_DIR = BASE_DIR / "results"
POST_DIR = DATA_DIR / "Post"
STAGE1_PATH = RESULTS_DIR / "uv_stage1.csv"
STAGE2_PATH = RESULTS_DIR / "uv_stage2.csv"
SURVEY_QUESTIONS_PATH = DATA_DIR / "survey_questions.csv"
SURVEY_RENAME_MAP_PATH = DATA_DIR / "survey_column_rename_stage3.csv"
POST_MAP_PATH = DATA_DIR / "post_survey_map.csv"
OUTPUT_PATH = RESULTS_DIR / "uv_open_ended.csv"

SURVEY_EXTRA_CODES = {"E21"}
POST_EXTRA_CODES = {"15", "18"}

def normalize_text(value: str) -> str:
    return re.sub(r"\s+", " ", str(value).strip()).lower()

def extract_column_code(column_name: str) -> str:
    if "_" not in column_name:
        return column_name.upper()
    return column_name.rsplit("_", 1)[-1].upper()

def load_survey_target_metadata(open_code_set: Set[str]) -> pd.DataFrame:
    rename_map = pd.read_csv(SURVEY_RENAME_MAP_PATH)
    rename_map = rename_map.dropna(
        subset=["group", "raw_column", "target_column", "question_code"]
    )
    rename_map["normalized_code"] = (
        rename_map["question_code"]
        .astype(str)
        .str.strip()
        .str.replace("question1", "E22", case=False)
        .str.upper()
    )
    metadata = rename_map.loc[
        rename_map["normalized_code"].isin(open_code_set),
        ["group", "raw_column", "target_column"],
    ].copy()
    metadata["raw_column"] = metadata["raw_column"].str.strip()
    metadata["target_column"] = metadata["target_column"].str.strip()
    metadata = metadata.dropna(subset=["raw_column", "target_column"])
    metadata = metadata.drop_duplicates()
    return metadata

def first_non_empty(series: pd.Series) -> object:
    for value in series:
        if pd.isna(value):
            continue
        if isinstance(value, str):
            stripped = value.strip()
            if not stripped:
                continue
            return stripped
        return value
    return pd.NA

def extract_stage2_raw_responses(target_metadata: pd.DataFrame) -> pd.DataFrame:
    if target_metadata.empty:
        return pd.DataFrame(columns=["respondent"])

    frames: List[pd.DataFrame] = []
    for group, group_rows in target_metadata.groupby("group"):
        survey_dir = (
            DATA_DIR
            / "Export"
            / f"Group {group}"
            / "Analyses"
            / f"Group {group}-2"
            / "Survey"
        )
        if not survey_dir.exists():
            continue
        group_rows = group_rows.copy()
        raw_columns = group_rows["raw_column"].tolist()
        rename_lookup = dict(zip(group_rows["raw_column"], group_rows["target_column"]))

        for path in sorted(survey_dir.glob("MERGED_SURVEY_RESPONSE_MATRIX-*.txt")):
            df_raw = pd.read_csv(
                path,
                sep="\t",
                engine="python",
                quoting=csv.QUOTE_NONE,
                dtype=str,
            )
            respondent_col = None
            for candidate in df_raw.columns:
                if normalize_text(candidate) in {
                    "respondent",
                    "respondent id",
                    "respondentid",
                }:
                    respondent_col = candidate
                    break
            if respondent_col is None:
                continue

            available_raw = [col for col in raw_columns if col in df_raw.columns]
            if not available_raw:
                continue

            subset_df = df_raw[[respondent_col, *available_raw]].copy()
            rename_subset = {raw: rename_lookup[raw] for raw in available_raw}
            subset_df = subset_df.rename(columns=rename_subset)
            subset_df = subset_df.rename(columns={respondent_col: "respondent"})
            subset_df["survey_open_source_path"] = str(path.relative_to(BASE_DIR))
            subset_df["respondent"] = pd.to_numeric(subset_df["respondent"], errors="coerce")
            subset_df = subset_df.dropna(subset=["respondent"])
            subset_df["respondent"] = subset_df["respondent"].astype(int)
            frames.append(subset_df)

    if not frames:
        return pd.DataFrame(columns=["respondent"])

    combined = pd.concat(frames, ignore_index=True)
    value_columns = [col for col in combined.columns if col != "respondent"]
    if not value_columns:
        return pd.DataFrame(columns=["respondent"])

    aggregated = (
        combined.groupby("respondent")[value_columns]
        .agg(first_non_empty)
        .reset_index()
        .sort_values("respondent")
    )
    return aggregated

def format_post_code(raw_code: float | str) -> str:
    if pd.isna(raw_code):
        raise ValueError("Open-ended question code is missing")
    if isinstance(raw_code, str):
        raw_code = raw_code.strip()
        if raw_code:
            try:
                numeric = float(raw_code)
                return format_post_code(numeric)
            except ValueError:
                return raw_code
        raise ValueError("Encountered blank question code")
    whole = int(raw_code)
    frac = raw_code - whole
    if abs(frac) < 1e-9:
        return f"Q{whole:02d}"
    sub = int(round(frac * 10))
    return f"Q{whole:02d}-{sub}"

def format_post_subscale(subscale: str | float, question: str) -> str:
    text = question.lower()
    if "what helped" in text:
        return "ComprehensionHelp"
    if "what would have improved" in text:
        return "ComprehensionImprove"
    if "what happened" in text:
        return "Recall"
    if isinstance(subscale, str) and subscale.strip():
        return subscale.strip().title().replace(" ", "")
    return "OpenEnded"

def build_stage2_open_ended(open_codes: Iterable[str]) -> pd.DataFrame:
    open_code_set = {code.upper() for code in open_codes}
    target_metadata = load_survey_target_metadata(open_code_set)

    open_file = pd.read_csv(RESULTS_DIR / "uv_stage2_open_ended.csv")
    source_col = "survey_file"
    metadata_cols = [
        col
        for col in open_file.columns
        if col.startswith("survey_") and col != source_col
    ]
    stage2_open = open_file.drop(columns=metadata_cols)
    if source_col in stage2_open.columns:
        stage2_open = stage2_open.rename(columns={source_col: "survey_open_source_path"})
        stage2_open["survey_open_source_path"] = stage2_open["survey_open_source_path"].astype(str).str.strip()
        stage2_open.loc[stage2_open["survey_open_source_path"] == "", "survey_open_source_path"] = pd.NA
    stage2_open["respondent"] = stage2_open["respondent"].astype(int)

    present_codes = {
        col.split("_")[-1].upper()
        for col in stage2_open.columns
        if col != "respondent"
    }
    missing_codes = open_code_set - present_codes

    if missing_codes:
        stage2 = pd.read_csv(STAGE2_PATH)
        extra_cols = [
            col
            for col in stage2.columns
            if (col != "respondent" and col.split("_")[-1].upper() in missing_codes)
        ]
        if extra_cols:
            extras = stage2[["respondent", *sorted(extra_cols)]]
            stage2_open = stage2_open.merge(extras, on="respondent", how="left")

    targets_needed = set(target_metadata["target_column"])
    existing_targets = {col for col in stage2_open.columns if col != "respondent"}
    null_targets = {
        col
        for col in targets_needed & existing_targets
        if stage2_open[col].notna().sum() == 0
    }
    missing_targets = targets_needed - existing_targets
    targets_to_backfill = missing_targets | null_targets

    if targets_to_backfill:
        raw_metadata = target_metadata[target_metadata["target_column"].isin(targets_to_backfill)]
        raw_df = extract_stage2_raw_responses(raw_metadata)
        if not raw_df.empty:
            stage2_open = stage2_open.set_index("respondent")
            raw_df = raw_df.set_index("respondent")
            stage2_open = stage2_open.reindex(stage2_open.index.union(raw_df.index))
            aligned_raw = raw_df.reindex(stage2_open.index)
            for target in targets_to_backfill:
                if target not in aligned_raw.columns:
                    continue
                values = aligned_raw[target]
                if target in stage2_open.columns:
                    stage2_open[target] = stage2_open[target].where(stage2_open[target].notna(), values)
                else:
                    stage2_open[target] = values
            if "survey_open_source_path" in aligned_raw.columns:
                path_values = aligned_raw["survey_open_source_path"]
                if "survey_open_source_path" in stage2_open.columns:
                    available = path_values.notna()
                    stage2_open.loc[available, "survey_open_source_path"] = path_values.loc[available]
                else:
                    stage2_open["survey_open_source_path"] = path_values
            stage2_open = stage2_open.reset_index()
            stage2_open["respondent"] = stage2_open["respondent"].astype(int)

    allowed_codes = {code.upper() for code in open_code_set}
    ordered_cols = ["respondent"]
    if "survey_open_source_path" in stage2_open.columns:
        ordered_cols.append("survey_open_source_path")
    ordered_cols.extend(
        col
        for col in stage2_open.columns
        if col not in ordered_cols and extract_column_code(col) in allowed_codes
    )
    stage2_open = stage2_open.loc[:, ordered_cols]

    return stage2_open

@dataclass(frozen=True)
class PostQuestion:
    group: str
    question: str
    title: str
    question_code: str
    subscale: str

def _normalize_question_code(value: float | str) -> str | None:
    if pd.isna(value):
        return None
    if isinstance(value, str):
        cleaned = value.strip()
        if cleaned.endswith(".0"):
            cleaned = cleaned[:-2]
        return cleaned
    if isinstance(value, (int, float)):
        if float(value).is_integer():
            return str(int(value))
        return str(value)
    return str(value)

def collect_allowed_post_codes(post_map: pd.DataFrame) -> Set[str]:
    allowed: Set[str] = set()
    for _, row in post_map.iterrows():
        raw_code = row.get("question_code")
        if pd.isna(raw_code):
            continue
        normalized = _normalize_question_code(raw_code)
        if normalized is None:
            continue
        is_open = str(row.get("type", "")).strip().lower() == "open-ended"
        is_extra = normalized in POST_EXTRA_CODES
        if not (is_open or is_extra):
            continue
        try:
            numeric = float(normalized)
            formatted = format_post_code(numeric)
        except ValueError:
            formatted = format_post_code(normalized)
        allowed.add(formatted.upper())
    return allowed

def explode_post_questions() -> List[PostQuestion]:
    post_map = pd.read_csv(POST_MAP_PATH)
    extra_codes = {code for code in POST_EXTRA_CODES if code}
    mask_open = post_map["type"].astype(str).str.strip().str.lower() == "open-ended"
    mask_extra = post_map["question_code"].apply(
        lambda value: _normalize_question_code(value) in extra_codes
    )
    open_ended = post_map.loc[mask_open | mask_extra].copy()
    questions: List[PostQuestion] = []
    for _, row in open_ended.iterrows():
        if pd.isna(row["question_code"]):
            continue
        code = format_post_code(row["question_code"])
        subscale = format_post_subscale(row.get("subscale", ""), row["question"])
        for group in ["A", "B", "C", "D", "E", "F"]:
            title = row.get(f"Group {group}")
            if isinstance(title, str) and title.strip():
                questions.append(
                    PostQuestion(
                        group=group,
                        question=row["question"],
                        title=title.strip(),
                        question_code=code,
                        subscale=subscale,
                    )
                )
    return questions

def load_post_open_ended(forms_map: Dict[str, Dict[str, str]]) -> pd.DataFrame:
    questions = explode_post_questions()
    grouped: Dict[str, List[PostQuestion]] = {}
    for q in questions:
        grouped.setdefault(q.group, []).append(q)

    frames: List[pd.DataFrame] = []
    for path in sorted(POST_DIR.glob("Group *_*.csv")):
        match = re.search(r"Group ([A-F])", path.name)
        if not match:
            continue
        group = match.group(1)
        if group not in grouped:
            continue
        df_raw = pd.read_csv(path)
        df_raw.columns = [col.strip() for col in df_raw.columns]
        normalized_lookup = {normalize_text(col): col for col in df_raw.columns}
        respondent_col = None
        for key, col in normalized_lookup.items():
            if "participant number" in key:
                respondent_col = col
                break
        if respondent_col is None:
            raise KeyError(f"Participant number column missing in {path}")
        respondent_series = df_raw[respondent_col].astype(str).str.extract(r"(\d+)")[0]
        df = pd.DataFrame({"respondent": pd.to_numeric(respondent_series, errors="coerce")})

        long_title = forms_map[group]["Long Form"]
        short_title = forms_map[group]["Short Form"]

        for question in grouped[group]:
            norm = normalize_text(question.question)
            raw_col = normalized_lookup.get(norm)
            if raw_col is None:
                continue
            if question.title == long_title:
                form = "Long"
            elif question.title == short_title:
                form = "Short"
            else:
                form = "All"
            column_name = f"{form}_{question.title}_Post_{question.subscale}_{question.question_code}"
            df[column_name] = df_raw[raw_col]

        df["post_open_source_path"] = str(path.relative_to(BASE_DIR))

        df = df.dropna(subset=["respondent"])
        df["respondent"] = df["respondent"].astype(int)
        frames.append(df)

    if not frames:
        return pd.DataFrame(columns=["respondent"])

    merged = pd.concat(frames, ignore_index=True)
    merged = merged.groupby("respondent", as_index=False).first()
    merged = merged.sort_values("respondent")
    return merged

def build_open_ended_table() -> pd.DataFrame:
    stage1 = pd.read_csv(STAGE1_PATH)
    stage1_forms = cast(
        Dict[str, Dict[str, str]],
        stage1[["group", "Short Form", "Long Form"]]
        .drop_duplicates()
        .set_index("group")
        .to_dict(orient="index")
    )

    survey_questions = pd.read_csv(SURVEY_QUESTIONS_PATH)
    open_codes = (
        survey_questions.loc[
            survey_questions["question_type"].astype(str).str.strip().str.lower() == "open ended",
            "question_code",
        ]
        .astype(str)
        .str.strip()
        .str.replace("question1", "E22", case=False)
    )
    open_codes = open_codes.tolist() + list(SURVEY_EXTRA_CODES)

    stage2_open = build_stage2_open_ended(open_codes)
    post_open = load_post_open_ended(stage1_forms)

    combined = stage1.merge(stage2_open, on="respondent", how="left")
    combined = combined.merge(post_open, on="respondent", how="left")

    allowed_survey_codes = (
        survey_questions.loc[
            survey_questions["question_type"].astype(str).str.strip().str.lower() == "open ended",
            "question_code",
        ]
        .astype(str)
        .str.strip()
        .str.replace("question1", "E22", case=False)
        .str.upper()
    )
    allowed_survey_codes = set(allowed_survey_codes).union({code.upper() for code in SURVEY_EXTRA_CODES})

    post_map = pd.read_csv(POST_MAP_PATH)
    allowed_post_codes = collect_allowed_post_codes(post_map)

    allowed_codes_total = allowed_survey_codes.union(allowed_post_codes)

    ordered_cols = list(stage1.columns)
    optional_cols: List[str] = []
    for col in combined.columns:
        if col in ordered_cols:
            continue
        if col in {"survey_open_source_path", "post_open_source_path"}:
            optional_cols.append(col)
            continue
        code = extract_column_code(col)
        if code in allowed_codes_total:
            optional_cols.append(col)

    combined = combined.loc[:, ordered_cols + sorted(optional_cols)]

    OUTPUT_PATH.parent.mkdir(parents=True, exist_ok=True)
    if OUTPUT_PATH.exists():
        try:
            OUTPUT_PATH.unlink()
        except PermissionError:
            temp_path = OUTPUT_PATH.with_suffix(".tmp")
            combined.to_csv(temp_path, index=False)
            try:
                temp_path.replace(OUTPUT_PATH)
                rel_path = OUTPUT_PATH.relative_to(BASE_DIR)
                print(
                    "Notice: replaced locked open-ended export via temporary file; "
                    f"wrote {rel_path}."
                )
                return combined
            except PermissionError:
                rel_temp = temp_path.relative_to(BASE_DIR)
                print(
                    "Warning: existing open-ended export is locked; "
                    f"wrote temporary file to {rel_temp} instead."
                )
                return combined

    combined.to_csv(OUTPUT_PATH, index=False)
    rel_path = OUTPUT_PATH.relative_to(BASE_DIR)
    print(f"Wrote {rel_path} with shape {combined.shape}.")
    return combined

uv_open = build_open_ended_table()
uv_open_path = OUTPUT_PATH
print(f"Stage 4 open-ended shape: {uv_open.shape}; saved to {uv_open_path.name}.")
uv_open.head(3)

Wrote results\uv_open_ended.csv with shape (83, 59).
Stage 4 open-ended shape: (83, 59); saved to uv_open_ended.csv.


,source_file,group,respondent,date_study,time_study,age,age_group,gender,ethnicity,income_group,...,Short_Mad Max_Survey_Enjoyment_E22,Short_The Town_Post_ComprehensionHelp_Q15-1,Short_The Town_Post_ComprehensionImprove_Q15-1,Short_The Town_Post_Comprehension_Q15,Short_The Town_Post_Recall_Q13,Short_The Town_Survey_Enjoyment_E16,Short_The Town_Survey_Enjoyment_E21,Short_The Town_Survey_Enjoyment_E22,post_open_source_path,survey_open_source_path
0,003_104.csv,A,104,10/16/2025,18:09:03,59,44-59,Male,White,"$60,000 or more per year",...,She was able to share her gift with people who...,None,None,None,None,NaN,NaN,<NA>,data\Post\Group A_ Post Viewing Questionnaire ...,data\Export\Group A\Analyses\Group A-2\Survey\...
1,002_106.csv,A,106,10/16/2025,19:35:05,30,28-43,Male,White,"$60,000 or more per year",...,I'm not usually a fan of horror. This scene br...,None,None,None,None,NaN,NaN,<NA>,data\Post\Group A_ Post Viewing Questionnaire ...,data\Export\Group A\Analyses\Group A-2\Survey\...
2,001_116.csv,A,116,10/18/2025,12:37:40,19,18-27,Male,White,"$35,000  $60,000 per year",...,It was from one of my favorite horror movies t...,None,None,None,None,NaN,NaN,<NA>,data\Post\Group F_ Post Viewing Questionnaire ...,data\Export\Group A\Analyses\Group A-2\Survey\...


In [36]:
# Stage 4 export → long format
from pathlib import Path

if "uv_open" not in globals():
    raise RuntimeError("Stage 4 table not found. Run the previous cell first.")
if "collect_allowed_post_codes" not in globals():
    raise RuntimeError("Stage 4 helper functions missing. Re-run the Stage 4 cell.")

survey_questions_fresh = pd.read_csv(SURVEY_QUESTIONS_PATH)
allowed_survey_codes = (
    survey_questions_fresh.loc[
        survey_questions_fresh["question_type"].astype(str).str.strip().str.lower() == "open ended",
        "question_code",
    ]
    .astype(str)
    .str.strip()
    .str.replace("question1", "E22", case=False)
    .str.upper()
)
allowed_survey_codes = set(allowed_survey_codes).union({code.upper() for code in SURVEY_EXTRA_CODES})

post_map_fresh = pd.read_csv(POST_MAP_PATH)
allowed_post_codes = collect_allowed_post_codes(post_map_fresh)
allowed_codes_total = allowed_survey_codes.union(allowed_post_codes)

value_columns = [
    col for col in uv_open.columns
    if ('_Survey_' in col or '_Post_' in col) and not col.lower().endswith('source_path')
]
value_columns = [
    col for col in value_columns if extract_column_code(col) in allowed_codes_total
]

if not value_columns:
    raise ValueError("No eligible open-ended response columns detected after filtering.")

def _parse_open_column(name: str):
    if '_Survey_' not in name and '_Post_' not in name:
        return None
    first_sep = name.find('_')
    if first_sep == -1:
        return None
    format_part = name[:first_sep]
    remainder = name[first_sep + 1 :]
    second_sep = remainder.find('_')
    if second_sep == -1:
        return None
    title_part = remainder[:second_sep]
    remainder2 = remainder[second_sep + 1 :]
    third_sep = remainder2.find('_')
    if third_sep == -1:
        return None
    questionnaire_part = remainder2[:third_sep]
    rest = remainder2[third_sep + 1 :]
    last_sep = rest.rfind('_')
    if last_sep == -1:
        question_part = ''
        question_code = rest
    else:
        question_part = rest[:last_sep]
        question_code = rest[last_sep + 1 :]
    return {
        'format': format_part,
        'title': title_part,
        'questionnaire': questionnaire_part,
        'question': question_part.replace('_', ' ').strip() or pd.NA,
        'question_code': question_code,
    }

long_df = uv_open.melt(
    id_vars=["respondent", "group"],
    value_vars=value_columns,
    var_name="column_name",
    value_name="response",
)

metadata_expanded = long_df["column_name"].apply(_parse_open_column)
metadata_df = pd.DataFrame(list(metadata_expanded))

combined_long = pd.concat([long_df, metadata_df], axis=1)
combined_long = combined_long.dropna(subset=["response", "question_code"])
combined_long["response"] = (
    combined_long["response"].astype(str).str.strip().replace({"": pd.NA, "nan": pd.NA})
)
combined_long = combined_long.dropna(subset=["response"])
combined_long["questionnaire"] = combined_long["questionnaire"].str.title()
combined_long["format"] = combined_long["format"].str.title()
combined_long["title"] = combined_long["title"].str.strip()
combined_long["question_code"] = combined_long["question_code"].str.strip()
combined_long["question"] = combined_long["question"].fillna("").str.replace("  ", " ").str.strip()
combined_long.loc[combined_long["question"] == "", "question"] = pd.NA

final_columns = [
    "respondent",
    "group",
    "questionnaire",
    "question_code",
    "question",
    "format",
    "title",
    "response",
]
open_long = combined_long[final_columns].reset_index(drop=True)

long_output_path = RESULTS_DIR / "uv_open_ended_long.csv"
open_long.to_csv(long_output_path, index=False)

print(
    "Wrote", long_output_path.relative_to(project_root),
    "with", open_long.shape[0], "rows and", open_long.shape[1], "columns."
)
open_long.head(10)

Wrote results\uv_open_ended_long.csv with 737 rows and 8 columns.


,respondent,group,questionnaire,question_code,question,format,title,response
0,116,A,Post,Q15-1,ComprehensionHelp,Long,Abbot Elementary,Seeing the actors talk about eating pizza and ...
1,107,F,Post,Q15-1,ComprehensionHelp,Long,Abbot Elementary,"The facial expressions, the emotions were writ..."
2,109,F,Post,Q15-1,ComprehensionHelp,Long,Abbot Elementary,That agree was avoiding answering what his fav...
3,11,F,Post,Q15-1,ComprehensionHelp,Long,Abbot Elementary,It was just a clear and funny scene. They were...
4,113,F,Post,Q15-1,ComprehensionHelp,Long,Abbot Elementary,It was reaction when he looked at the pizza an...
5,17,F,Post,Q15-1,ComprehensionHelp,Long,Abbot Elementary,The dialogue between characters and my own abi...
6,41,F,Post,Q15-1,ComprehensionHelp,Long,Abbot Elementary,Would helped my understanding is that I got in...
7,42,F,Post,Q15-1,ComprehensionHelp,Long,Abbot Elementary,"The scene was well written, and even though I ..."
8,44,F,Post,Q15-1,ComprehensionHelp,Long,Abbot Elementary,Because Greg was shown to be uncomfortable wit...
9,50,F,Post,Q15-1,ComprehensionHelp,Long,Abbot Elementary,Facial clues and body movements help me with m...


## Stage 5.1: LLM-Based Recall Scoring

This stage applies automated qualitative scoring to open-ended recall responses using GPT-4.1. The workflow:
- Loads model event sequences from `data/model_answers_events.md` for each title/format combination
- Reads long-form recall responses from `results/uv_open_ended_long_recall.csv` (Stage 4 output)
- Batches responses and calls the OpenAI Responses API to generate recall quality scores (0-100), confidence scores (0-100), and brief rationales
- Exports scored responses to `results/recall_coded_responses_full.csv` for downstream merging into the UV

**Prerequisites**: Valid `OPENAI_API_KEY` environment variable and `data/model_answers_events.md` file.

In [37]:
import json
import os
import re
import textwrap
from io import StringIO
from typing import Dict, List, Optional, Tuple

import pandas as pd

try:
    from openai import OpenAI
except ImportError:
    OpenAI = None

# Configuration
MODEL_EVENTS_PATH = project_root / "data" / "model_answers_events.md"
OPEN_ENDED_RECALL_PATH = RESULTS_DIR / "uv_open_ended_long_recall.csv"
RECALL_OUTPUT_PATH = RESULTS_DIR / "recall_coded_responses_full.csv"
KEY_MOMENT_ERROR_LOG_PATH = RESULTS_DIR / "recall_coded_responses_errors.csv"
MODEL_NAME = "gpt-4.1"
BATCH_SIZE = 3

# Initialize OpenAI client
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
openai_client = OpenAI(api_key=OPENAI_API_KEY) if (OpenAI and OPENAI_API_KEY) else None

if not MODEL_EVENTS_PATH.exists():
    raise FileNotFoundError(
        f"Model events file not found: {MODEL_EVENTS_PATH}. "
        "Please ensure data/model_answers_events.md exists."
    )

if not OPEN_ENDED_RECALL_PATH.exists():
    raise FileNotFoundError(
        f"Open-ended recall data not found: {OPEN_ENDED_RECALL_PATH}. "
        "Stage 4 must complete before running recall scoring."
    )

print(f"Loading model events from: {MODEL_EVENTS_PATH.name}")
print(f"Loading recall responses from: {OPEN_ENDED_RECALL_PATH.name}")
print(f"OpenAI client available: {openai_client is not None}")

Loading model events from: model_answers_events.md
Loading recall responses from: uv_open_ended_long_recall.csv
OpenAI client available: True


In [38]:
# Stage 5 helper functions shared by Stage 5.1 and Stage 5.2
from wbdlib import (
    SYSTEM_PROMPT_STAGE51,
    build_batch_prompt,
    call_llm_batch,
    describe_event_source,
    enrich_dataframe_with_scores,
    normalise_form,
    normalise_title,
    parse_llm_json,
    parse_model_events,
    resolve_event_list,
 )

print("Stage 5 helper functions imported from wbdlib.")

Stage 5 helper functions imported from wbdlib.


In [39]:
# Load and parse model events
model_events_text = MODEL_EVENTS_PATH.read_text(encoding="utf-8")
model_events_lookup = parse_model_events(model_events_text)
print(f"Loaded events for {len(model_events_lookup)} title/format combinations")
print("Parsed combinations:")
for title_key, form_key in sorted(model_events_lookup.keys()):
    print(f"  - Title: {title_key} | Form: {form_key}")

# Load recall responses
try:
    recall_df = pd.read_csv(OPEN_ENDED_RECALL_PATH)
except UnicodeDecodeError:
    with OPEN_ENDED_RECALL_PATH.open("r", encoding="cp1252", errors="ignore") as fh:
        buffer = StringIO(fh.read())
    recall_df = pd.read_csv(buffer)

# Normalise column names for downstream logic
recall_df.columns = [col.strip().lower() for col in recall_df.columns]
column_aliases = {
    "format": "form",
    "media_format": "form",
    "media_form": "form",
    "content_form": "form",
}
for candidate, target in column_aliases.items():
    if candidate in recall_df.columns and target not in recall_df.columns:
        recall_df = recall_df.rename(columns={candidate: target})
if "form" not in recall_df.columns:
    available = ", ".join(sorted(recall_df.columns))
    raise KeyError(f"Expected a 'form' column in recall data; available columns: {available}")

if "id" not in recall_df.columns:
    recall_df.insert(0, "id", range(len(recall_df)))

print(f"Loaded {len(recall_df)} recall responses")
print(f"Unique titles: {recall_df['title'].nunique()} | Forms: {sorted(recall_df['form'].unique().tolist())}")
recall_df.head(3)

Loaded events for 6 title/format combinations
Parsed combinations:
  - Title: abbot elementary | Form: long
  - Title: abbot elementary | Form: short
  - Title: mad max | Form: long
  - Title: mad max | Form: short
  - Title: the town | Form: long
  - Title: the town | Form: short
Loaded 162 recall responses
Unique titles: 3 | Forms: ['Long', 'Short']


,id,respondent,group,questionnaire,question_code,question,form,title,response
0,0,116,A,Post,Q13,Recall,Long,Abbot Elementary,The black male actor lied about liking pizza a...
1,1,107,F,Post,Q13,Recall,Long,Abbot Elementary,The guy grabbing the pizza in reality doesn’t ...
2,2,109,F,Post,Q13,Recall,Long,Abbot Elementary,"Greg, Jacob and staff were having lunch in the..."


In [40]:
# parse_llm_json helper provided by wbdlib
from wbdlib import parse_llm_json

print("parse_llm_json helper imported from wbdlib.")

parse_llm_json helper imported from wbdlib.


In [41]:
# Stage 5.1 sanity summary (deduplicated)
if 'recall_df' not in globals():
    raise RuntimeError("recall_df is not defined. Run the earlier Stage 5.1 setup cells first.")
if 'model_events_lookup' not in globals():
    raise RuntimeError("model_events_lookup is not defined. Run the event parsing cell first.")

print(f"Recall dataframe shape: {recall_df.shape}")
print(f"Recall columns: {sorted(recall_df.columns.tolist())}")
print(f"Unique titles: {sorted(recall_df['title'].unique().tolist())}")
print(f"Forms present: {sorted(recall_df['form'].unique().tolist())}")
print(f"Total model event combos: {len(model_events_lookup)}")

Recall dataframe shape: (162, 9)
Recall columns: ['form', 'group', 'id', 'question', 'question_code', 'questionnaire', 'respondent', 'response', 'title']
Unique titles: ['Abbot Elementary', 'Mad Max', 'The Town']
Forms present: ['Long', 'Short']
Total model event combos: 6


In [42]:
# Execute Method 2 recall scoring and persist outputs
if "recall_df" not in globals():
    raise RuntimeError("recall_df is not defined. Re-run the Stage 5.1 setup cells first.")
if "model_events_lookup" not in globals():
    raise RuntimeError("model_events_lookup is not defined. Re-run the event parsing cell first.")
if openai_client is None:
    raise RuntimeError("OPENAI_API_KEY is not configured. Set it before running Stage 5.1 recall scoring.")

if recall_df.empty:
    raise ValueError("Recall dataframe is empty; nothing to score.")

print(f"Scoring {len(recall_df)} recall responses in batches of {BATCH_SIZE} using {MODEL_NAME}.")

all_results: List[Dict[str, object]] = []
missing_event_keys: set[Tuple[str, str]] = set()

for batch_index, start in enumerate(range(0, len(recall_df), BATCH_SIZE), start=1):
    batch_df = recall_df.iloc[start : start + BATCH_SIZE]
    prompt_text, missing_keys = build_batch_prompt(batch_df, model_events_lookup)
    if missing_keys:
        missing_event_keys.update(missing_keys)

    raw_response = call_llm_batch(
        prompt_text,
        client_obj=openai_client,
        model=MODEL_NAME,
    )
    batch_results = parse_llm_json(raw_response)

    # Normalise data types for downstream merging
    for entry in batch_results:
        entry["id"] = int(entry["id"])
        entry["recall_score"] = int(entry["recall_score"])
        entry["confidence_score"] = int(entry["confidence_score"])

    all_results.extend(batch_results)
    print(f"  ✓ Batch {batch_index} scored ({len(batch_results)} rows)")

if missing_event_keys:
    print("Warning: Missing model events for the following title/form combinations:")
    for title_key, form_key in sorted(missing_event_keys):
        print(f"  - Title: {title_key} | Form: {form_key}")

if not all_results:
    raise RuntimeError("No scores were returned by the LLM. Aborting Stage 5.1 output.")

scored_recall_df = enrich_dataframe_with_scores(recall_df, all_results)
scored_recall_df = scored_recall_df.sort_values(["respondent", "title", "form", "id"]).reset_index(drop=True)

score_columns = ["recall_score", "confidence_score"]
valid_scores = scored_recall_df.dropna(subset=score_columns)
missing_count = len(scored_recall_df) - len(valid_scores)

scored_recall_df.to_csv(RECALL_OUTPUT_PATH, index=False)
print("\nRecall scoring complete.")
print(f"  Output saved to: {RECALL_OUTPUT_PATH.relative_to(project_root)}")
print(f"  Total rows: {len(scored_recall_df)}")
print(f"  Scored rows: {len(valid_scores)}")
print(f"  Rows missing scores: {missing_count}")

summary = valid_scores[score_columns].describe()
print("\nScore distribution (valid rows):")
print(summary)

scored_recall_df.head(5)

Scoring 162 recall responses in batches of 3 using gpt-4.1.
  ✓ Batch 1 scored (3 rows)
  ✓ Batch 2 scored (3 rows)


KeyboardInterrupt: 

## Stage 5.2: Key-Moment Recall Scoring

This stage re-scores the open-ended recall responses using the short-form key-moment event lists. Long-form respondents are evaluated against the short-form benchmark so that recall metrics are comparable across formats. The harness reuses the Stage 5 helper functions, prefers short-form events when available, and writes `results/recall_coded_responses_key_moment.csv`. Run these cells only after the Stage 5.2 prompt and pilot have been approved.

In [ ]:
KEY_MOMENT_OUTPUT_PATH = RESULTS_DIR / "recall_coded_responses_key_moment.csv"
KEY_MOMENT_ERROR_LOG_PATH = RESULTS_DIR / "recall_coded_responses_key_moment_errors.csv"
SYSTEM_PROMPT_STAGE52 = textwrap.dedent(
    """
    You are an expert qualitative coder focusing on key-moment recall for media research.
    Scoring guidelines:
    - The MODEL EVENTS describe the short-form key moment. Apply them even when the participant viewed the long-form cut.
    - Compare the participant response to the key-moment events and assess number of events recalled, accuracy, specificity, and ordering.
    - If the participant explicitly says they do not remember, set recall_score to 0 and confidence_score to at least 90.
    - Ignore long-form-only details that are not part of the key moment when assigning the score.
    Return a valid JSON array containing one object per response with fields:
      • "id" (integer row identifier)
      • "recall_score" (integer 0-100)
      • "confidence_score" (integer 0-100)
      • "rationale" (1-3 sentences referencing the MODEL EVENTS)
    Do not include any commentary outside of the JSON array.
    """
).strip()
print("Stage 5.2 configuration ready. Review SYSTEM_PROMPT_STAGE52 and KEY_MOMENT_ERROR_LOG_PATH before running the scoring cell.")

Stage 5.2 configuration ready. Review SYSTEM_PROMPT_STAGE52 before running the scoring cell.


In [ ]:
# Stage 5.2 key-moment scoring (execute only after approval)
if "recall_df" not in globals():
    raise RuntimeError("recall_df is not defined. Run the Stage 5.1 setup cells first.")
if "model_events_lookup" not in globals():
    raise RuntimeError("model_events_lookup is not defined. Run the Stage 5.1 event parsing cell first.")
if openai_client is None:
    raise RuntimeError("OPENAI_API_KEY is not configured. Set it before running Stage 5.2 key-moment scoring.")
if recall_df.empty:
    raise ValueError("Recall dataframe is empty; nothing to score.")

print(f"Scoring {len(recall_df)} recall responses in batches of {BATCH_SIZE} using {MODEL_NAME}.")
print("Long-form respondents will be evaluated against short-form key-moment events.")

all_keymoment_results: List[Dict[str, object]] = []
missing_keymoment_events: set[Tuple[str, str]] = set()
key_moment_metadata_frames: List[pd.DataFrame] = []
error_records: List[Dict[str, object]] = []

for batch_index, start in enumerate(range(0, len(recall_df), BATCH_SIZE), start=1):
    batch_df = recall_df.iloc[start : start + BATCH_SIZE]
    expected_ids = [int(value) for value in batch_df["id"].tolist()]

    prompt_result = build_batch_prompt(
        batch_df,
        model_events_lookup,
        prefer_short_for_long=True,
        include_metadata=True,
    )
    if len(prompt_result) == 3:
        prompt_text, missing_keys, metadata_df = prompt_result
    else:
        prompt_text, missing_keys = prompt_result
        metadata_df = pd.DataFrame(
            columns=[
                "id",
                "respondent",
                "title",
                "form_requested",
                "event_form_used",
                "event_count",
                "event_label",
            ]
        )
    if missing_keys:
        missing_keymoment_events.update(missing_keys)
    if metadata_df is None or metadata_df.empty:
        metadata_df = pd.DataFrame(
            columns=[
                "id",
                "respondent",
                "title",
                "form_requested",
                "event_form_used",
                "event_count",
                "event_label",
            ]
        )
    metadata_df = metadata_df.copy()
    metadata_df["batch_index"] = batch_index
    metadata_df["rerun"] = False
    key_moment_metadata_frames.append(metadata_df)

    raw_response = call_llm_batch(
        prompt_text,
        client_obj=openai_client,
        model=MODEL_NAME,
        system_prompt=SYSTEM_PROMPT_STAGE52,
    )
    batch_results_raw = parse_llm_json(raw_response)

    result_by_id: Dict[int, Dict[str, object]] = {}
    invalid_ids: set[int] = set()

    for entry in batch_results_raw:
        rid_raw = entry.get("id")
        try:
            rid = int(rid_raw)
            recall_val = int(entry.get("recall_score"))
            conf_val = int(entry.get("confidence_score"))
        except (TypeError, ValueError):
            try:
                rid_context = int(rid_raw) if rid_raw is not None else None
            except (TypeError, ValueError):
                rid_context = None
            if rid_context is not None:
                invalid_ids.add(rid_context)
            continue
        entry["id"] = rid
        entry["recall_score"] = recall_val
        entry["confidence_score"] = conf_val
        result_by_id[rid] = entry

    missing_ids: set[int] = {rid for rid in expected_ids if rid not in result_by_id}
    missing_ids.update(invalid_ids & set(expected_ids))

    for missing_id in sorted(missing_ids):
        row_df = batch_df.loc[batch_df["id"] == missing_id]
        if row_df.empty:
            continue
        rerun_result = build_batch_prompt(
            row_df,
            model_events_lookup,
            prefer_short_for_long=True,
            include_metadata=True,
        )
        if len(rerun_result) == 3:
            single_prompt, single_missing_keys, single_metadata = rerun_result
        else:
            single_prompt, single_missing_keys = rerun_result
            single_metadata = pd.DataFrame(
                columns=[
                    "id",
                    "respondent",
                    "title",
                    "form_requested",
                    "event_form_used",
                    "event_count",
                    "event_label",
                ]
            )
        if single_missing_keys:
            missing_keymoment_events.update(single_missing_keys)
        if single_metadata is None or single_metadata.empty:
            single_metadata = pd.DataFrame(
                columns=[
                    "id",
                    "respondent",
                    "title",
                    "form_requested",
                    "event_form_used",
                    "event_count",
                    "event_label",
                ]
            )
        single_metadata = single_metadata.copy()
        single_metadata["batch_index"] = batch_index
        single_metadata["rerun"] = True
        key_moment_metadata_frames.append(single_metadata)

        rerun_success = False
        rerun_reason: Optional[str] = None

        try:
            single_raw = call_llm_batch(
                single_prompt,
                client_obj=openai_client,
                model=MODEL_NAME,
                system_prompt=SYSTEM_PROMPT_STAGE52,
            )
            single_results = parse_llm_json(single_raw)
            for entry in single_results:
                rid_raw = entry.get("id")
                try:
                    rid = int(rid_raw)
                    recall_val = int(entry.get("recall_score"))
                    conf_val = int(entry.get("confidence_score"))
                except (TypeError, ValueError):
                    continue
                if rid == missing_id:
                    entry["id"] = rid
                    entry["recall_score"] = recall_val
                    entry["confidence_score"] = conf_val
                    result_by_id[rid] = entry
                    rerun_success = True
                    break
        except Exception as exc:
            rerun_reason = str(exc)

        if rerun_success:
            print(f"  ↻ Rerun succeeded for id {missing_id}")
            continue

        if rerun_reason is None:
            rerun_reason = "LLM returned no valid result after rerun"
        row_info = row_df.iloc[0]
        error_records.append(
            {
                "id": int(missing_id),
                "respondent": row_info.get("respondent", pd.NA),
                "title": row_info.get("title", pd.NA),
                "form": row_info.get("form", pd.NA),
                "batch_index": batch_index,
                "rerun_attempted": True,
                "reason": rerun_reason,
            }
        )
        print(f"  ⚠ Rerun failed for id {missing_id}: {rerun_reason}")

    ordered_results = [result_by_id[rid] for rid in expected_ids if rid in result_by_id]
    all_keymoment_results.extend(ordered_results)

    unresolved_count = len(expected_ids) - len(ordered_results)
    print(f"  ✓ Batch {batch_index} completed ({len(ordered_results)} rows)")
    if unresolved_count:
        print(f"    ⚠ {unresolved_count} rows remain unresolved after rerun (see error log)")

if missing_keymoment_events:
    print("Warning: Missing model events for the following title/form combinations:")
    for title_key, form_key in sorted(missing_keymoment_events):
        print(f"  - Title: {title_key} | Form: {form_key}")

if not all_keymoment_results:
    raise RuntimeError("No scores were returned by the LLM. Aborting Stage 5.2 output.")

scored_keymoment_df = enrich_dataframe_with_scores(recall_df, all_keymoment_results)
scored_keymoment_df = scored_keymoment_df.sort_values(["respondent", "title", "form", "id"]).reset_index(drop=True)

score_columns = ["recall_score", "confidence_score"]
valid_keymoment_scores = scored_keymoment_df.dropna(subset=score_columns)
missing_keymoment_count = len(scored_keymoment_df) - len(valid_keymoment_scores)

scored_keymoment_df.to_csv(KEY_MOMENT_OUTPUT_PATH, index=False)
print("\nKey-moment scoring complete.")
print(f"  Output saved to: {KEY_MOMENT_OUTPUT_PATH.relative_to(project_root)}")
print(f"  Total rows: {len(scored_keymoment_df)}")
print(f"  Scored rows: {len(valid_keymoment_scores)}")
print(f"  Rows missing scores: {missing_keymoment_count}")

key_moment_event_metadata_columns = [
    "id",
    "respondent",
    "title",
    "form_requested",
    "event_form_used",
    "event_count",
    "event_label",
    "batch_index",
    "rerun",
]
if key_moment_metadata_frames:
    key_moment_event_metadata = pd.concat(key_moment_metadata_frames, ignore_index=True)
    for col in key_moment_event_metadata_columns:
        if col not in key_moment_event_metadata.columns:
            key_moment_event_metadata[col] = pd.NA
    key_moment_event_metadata = key_moment_event_metadata[key_moment_event_metadata_columns]
else:
    key_moment_event_metadata = pd.DataFrame(columns=key_moment_event_metadata_columns)

if not key_moment_event_metadata.empty:
    usage_summary = (
        key_moment_event_metadata.groupby(["form_requested", "event_form_used", "rerun"])
        .size()
        .rename("count")
        .reset_index()
    )
    print("\nEvent form usage summary:")
    print(usage_summary.to_string(index=False))
else:
    print("\nNo event metadata generated")

if error_records:
    error_df = pd.DataFrame(error_records)
    error_df.to_csv(KEY_MOMENT_ERROR_LOG_PATH, index=False)
    print(
        "\n⚠ {count} responses could not be scored even after rerun. Details saved to {path}".format(
            count=len(error_df),
            path=KEY_MOMENT_ERROR_LOG_PATH.relative_to(project_root),
        )
    )
else:
    if KEY_MOMENT_ERROR_LOG_PATH.exists():
        try:
            KEY_MOMENT_ERROR_LOG_PATH.unlink()
        except OSError:
            pass
    print("\nNo unresolved scoring errors encountered.")

summary_keymoment = valid_keymoment_scores[score_columns].describe()
print("\nKey-moment score distribution (valid rows):")
print(summary_keymoment)

scored_keymoment_df.head(5)

Scoring 162 recall responses in batches of 3 using gpt-4.1.
Long-form respondents will be evaluated against short-form key-moment events.
  ✓ Batch 1 completed (3 rows)
  ✓ Batch 1 completed (3 rows)
  ✓ Batch 2 completed (3 rows)
  ✓ Batch 2 completed (3 rows)
  ✓ Batch 3 completed (3 rows)
  ✓ Batch 3 completed (3 rows)
  ✓ Batch 4 completed (3 rows)
  ✓ Batch 4 completed (3 rows)
  ✓ Batch 5 completed (3 rows)
  ✓ Batch 5 completed (3 rows)
  ✓ Batch 6 completed (3 rows)
  ✓ Batch 6 completed (3 rows)
  ✓ Batch 7 completed (3 rows)
  ✓ Batch 7 completed (3 rows)
  ✓ Batch 8 completed (3 rows)
  ✓ Batch 8 completed (3 rows)
  ✓ Batch 9 completed (3 rows)
  ✓ Batch 9 completed (3 rows)
  ✓ Batch 10 completed (3 rows)
  ✓ Batch 10 completed (3 rows)
  ✓ Batch 11 completed (3 rows)
  ✓ Batch 11 completed (3 rows)
  ✓ Batch 12 completed (3 rows)
  ✓ Batch 12 completed (3 rows)
  ✓ Batch 13 completed (3 rows)
  ✓ Batch 13 completed (3 rows)
  ✓ Batch 14 completed (3 rows)
  ✓ Batch 14 com

NameError: name 'KEY_MOMENT_ERROR_LOG_PATH' is not defined

## UV Merge
This consolidation step reloads the Stage 2 and Stage 3 outputs, checks their respondent-level metadata against the Stage 1 base, and publishes a merged UV file plus an issues log highlighting any discrepancies.

**Stage 5.1 Integration**: If `recall_coded_responses_full.csv` exists (generated by Stage 5.1 LLM recall scoring), recall scores are pivoted and merged into the UV using respondent+title+form as keys, creating columns named `{Form}_{Title}_Post_Recall_OpenEndedSum`.
**Stage 5.2 Integration**: If `recall_coded_responses_key_moment.csv` exists (generated by Stage 5.2 key-moment scoring), the same merge logic adds `{Form}_{Title}_Post_Recall_OpenEndedKMS` columns so analysts can compare like-for-like recall across formats.

In [2]:
# UV Merge with recall-coded open-ended scores (Stage 5.1 + Stage 5.2)
import numpy as np
import pandas as pd
from pathlib import Path

results_dir = project_root / "results"
stage1_path = results_dir / "uv_stage1.csv"
stage2_path = results_dir / "uv_stage2.csv"
stage3_path = results_dir / "uv_stage3.csv"

for required_path in [stage1_path, stage2_path, stage3_path]:
    if not required_path.exists():
        raise FileNotFoundError(f"Required file not found: {required_path}")

uv_stage1_base = pd.read_csv(stage1_path)
uv_stage2 = pd.read_csv(stage2_path)
uv_stage3 = pd.read_csv(stage3_path)

for df in (uv_stage1_base, uv_stage2, uv_stage3):
    df["respondent"] = df["respondent"].astype(str).str.strip()

baseline_cols = list(uv_stage1_base.columns)
baseline_set = set(baseline_cols)

def _load_recall_matrix(
    candidate_paths: list[Path],
    *,
    stage_label: str,
    column_suffix: str,
    ) -> pd.DataFrame:
    """Load a recall scoring file, pivot to wide format, and normalise respondent IDs."""
    path = next((p for p in candidate_paths if p.exists()), None)
    if path is None:
        print(f"Notice: {stage_label} file not found; skipping {stage_label.lower()} merge.")
        return pd.DataFrame(columns=["respondent"])

    suffix = path.suffix.lower()
    print(f"Loading {stage_label} scores from: {path.name}")

    if suffix == ".csv":
        coded = pd.read_csv(path)
    elif suffix in {".xlsx", ".xls"}:
        coded = pd.read_excel(path)
    elif suffix == ".parquet":
        coded = pd.read_parquet(path)
    else:
        raise ValueError(f"Unsupported {stage_label} extension: {path.suffix}")

    coded.columns = [str(col).strip().lower() for col in coded.columns]
    required_columns = {"respondent", "form", "title", "recall_score"}
    missing = required_columns - set(coded.columns)
    if missing:
        raise KeyError(
            f"{stage_label} is missing required columns: " + ", ".join(sorted(missing))
        )

    coded = coded.copy()
    coded["respondent"] = pd.to_numeric(coded["respondent"], errors="coerce")
    coded = coded.dropna(subset=["respondent"])
    coded["respondent"] = coded["respondent"].astype(int).astype(str)
    coded["form"] = coded["form"].astype(str).str.title().str.strip()
    coded["title"] = coded["title"].astype(str).str.strip()

    records: list[dict[str, object]] = []
    for row in coded.itertuples(index=False):
        sum_name = f"{row.form}_{row.title}_{column_suffix}"
        records.append({
            "respondent": row.respondent,
            "column_name": sum_name,
            "value": getattr(row, "recall_score"),
        })

    if not records:
        print(f"Notice: {stage_label} produced no rows after normalization; skipping merge.")
        return pd.DataFrame(columns=["respondent"])

    wide = (
        pd.DataFrame(records)
        .pivot(index="respondent", columns="column_name", values="value")
        .reset_index()
    )
    wide.columns.name = None
    wide["respondent"] = wide["respondent"].astype(int).astype(str)

    return wide

stage51_candidates = [
    results_dir / "recall_coded_responses_full.csv",
    results_dir / "recall_coded_responses.csv",
    results_dir / "coded_responses_full.csv",
    results_dir / "coded_responses_full.parquet",
    project_root / "data" / "coded_responses_full.csv",
    project_root / "data" / "coded_responses_full.parquet",
    project_root / "recall_openended" / "coded_responses_full.csv",
    project_root / "recall_openended" / "coded_responses_full.parquet",
]
stage51_wide = _load_recall_matrix(
    stage51_candidates, stage_label="Stage 5.1", column_suffix="Post_Recall_OpenEndedSum"
 )

stage52_candidates = [
    results_dir / "recall_coded_responses_key_moment.csv",
    results_dir / "recall_coded_responses_key_moment.parquet",
    project_root / "recall_openended" / "key moment" / "recall_coded_responses_key_moment.csv",
    project_root / "recall_openended" / "key moment" / "recall_coded_responses_key_moment.parquet",
]
stage52_wide = _load_recall_matrix(
    stage52_candidates, stage_label="Stage 5.2", column_suffix="Post_Recall_OpenEndedKMS"
 )

def _report_new_columns(wide: pd.DataFrame, label: str) -> set[str]:
    if wide.empty:
        return set()
    candidate_cols = set(wide.columns) - {"respondent"}
    new_cols = sorted(candidate_cols - baseline_set)
    if new_cols:
        print(f"Adding {label} columns to UV:")
        for col in new_cols:
            print(f"  - {col}")
    baseline_set.update(candidate_cols)
    return candidate_cols

_ = _report_new_columns(stage51_wide, "Stage 5.1 recall")
_ = _report_new_columns(stage52_wide, "Stage 5.2 key-moment recall")

uv = uv_stage1_base.merge(uv_stage2, on="respondent", how="left")
uv = uv.merge(uv_stage3, on="respondent", how="left")

if not stage51_wide.empty:
    uv = uv.merge(stage51_wide, on="respondent", how="left")
if not stage52_wide.empty:
    uv = uv.merge(stage52_wide, on="respondent", how="left")

uv_output_path = results_dir / "uv_merged.csv"
uv.to_csv(uv_output_path, index=False)
print(f"Merged UV exported to {uv_output_path.relative_to(project_root)}")

Loading Stage 5.1 scores from: recall_coded_responses_full.csv
Loading Stage 5.2 scores from: recall_coded_responses_key_moment.csv
Adding Stage 5.1 recall columns to UV:
  - Long_Abbot Elementary_Post_Recall_OpenEndedSum
  - Long_Mad Max_Post_Recall_OpenEndedSum
  - Long_The Town_Post_Recall_OpenEndedSum
  - Short_Abbot Elementary_Post_Recall_OpenEndedSum
  - Short_Mad Max_Post_Recall_OpenEndedSum
  - Short_The Town_Post_Recall_OpenEndedSum
Adding Stage 5.2 key-moment recall columns to UV:
  - Long_Abbot Elementary_Post_Recall_OpenEndedKMS
  - Long_Mad Max_Post_Recall_OpenEndedKMS
  - Long_The Town_Post_Recall_OpenEndedKMS
  - Short_Abbot Elementary_Post_Recall_OpenEndedKMS
  - Short_Mad Max_Post_Recall_OpenEndedKMS
  - Short_The Town_Post_Recall_OpenEndedKMS
Merged UV exported to results\uv_merged.csv


## Client Deliverables
This section will curate the tables, exports, and visual summaries required for the client handoff. Populate placeholder cells below with the final deliverable generation workflows once the requirements are finalized.

### Client Deliverables extraction blueprint
1. Load post-study raw CSVs (`data/Post/*`) with `load_post_survey_raw`, normalizing headers and capturing `source_path` plus submitted timestamps.
2. Reshape the raw grid to long format via `reshape_post_survey_long`, generating one row per (`respondent`, `question_code`) while cleaning respondent IDs (string coercion, zero-padding, whitespace trim) and standardizing `questionnaire='Post'`.
3. Join metadata using `attach_post_metadata`, merging `post_survey_map.csv` on `question_code` to append `question`, `type`, `subscale`, `category`, `accuracy`, and deriving `stimulus_form`/`stimulus_title` by mapping group-specific entries against Stage 1 exposures (`Short Form`, `Long Form`).
4. Enrich with scores through `append_post_scores`:
   - Melt Stage 3 recognition metrics to long form, parse column tokens to recover `question_code`, and attach binary/likert numeric scores plus confidence.
   - Merge Stage 5 recall outputs (`results/recall_coded_responses_full.csv`, `..._key_moment.csv`) to reuse existing LLM `recall_score`, `confidence_score`, and `rationale` as `score_value`, `score_confidence`, and `score_explanation`.
5. Finalize dataset by computing `response_clean`/`response_numeric` (numeric parsing for likert/binary, leave multiples as lists), tagging questions without scores, and persisting `results/post_study_survey_responses.csv`.

In [3]:
# Client Deliverables helpers
import math
import numpy as np
import pandas as pd
from pathlib import Path
import re
from typing import Dict, Iterable

project_root = globals().get("project_root", Path.cwd().parent)
DATA_DIR = project_root / "data"
RESULTS_DIR = project_root / "results"
POST_DIR = DATA_DIR / "Post"

TITLE_NORMALIZATION = {
    "abbott elementary": "Abbot Elementary",
    "abbot elementary": "Abbot Elementary",
    "schitts creek": "Schittss Creek",
    "schittss creek": "Schittss Creek",
    "mad max fury road": "Mad Max",
    "mad max": "Mad Max",
    "the town (long)": "The Town",
}

YES_VALUES = {"yes", "y", "true", "1"}
NO_VALUES = {"no", "n", "false", "0"}
LIKERT_PATTERN = re.compile(r"^\s*([0-9]+(?:\.[0-9]+)?)")
CODE_PATTERN = re.compile(r"^\s*([0-9]+(?:\.[0-9]+)*)")

def canonicalize_title(raw_title: str | float | None) -> str:
    if pd.isna(raw_title):
        return ""
    text = str(raw_title).strip()
    if not text:
        return ""
    return TITLE_NORMALIZATION.get(text.lower(), text)

def normalize_question_text(text: str | float | None) -> str:
    if pd.isna(text):
        return ""
    normalized = re.sub(r"\s+", " ", str(text)).strip().lower()
    normalized = re.sub(r"\.(\d+)$", "", normalized)
    return normalized

def merge_duplicate_columns(df: pd.DataFrame) -> pd.DataFrame:
    merged: Dict[str, pd.Series] = {}
    column_order: list[str] = []
    for column in df.columns:
        normalized = re.sub(r"\s+", " ", str(column)).strip()
        normalized = re.sub(r"\.(\d+)$", "", normalized)
        series = df[column].astype("object")
        series = series.where(series.notna(), np.nan)
        series = series.map(lambda v: v.strip() if isinstance(v, str) else v)
        if normalized not in merged:
            merged[normalized] = series
            column_order.append(normalized)
        else:
            merged[normalized] = merged[normalized].combine_first(series)
    return pd.DataFrame({name: merged[name] for name in column_order})

def extract_question_code(text: str | float | None) -> str | None:
    if pd.isna(text):
        return None
    match = CODE_PATTERN.match(str(text))
    return match.group(1) if match else None

def clean_response(value):
    if pd.isna(value):
        return np.nan
    if isinstance(value, str):
        cleaned = value.strip()
        return cleaned if cleaned else np.nan
    return value

def parse_binary(value):
    if pd.isna(value):
        return np.nan
    text = str(value).strip().lower()
    if not text:
        return np.nan
    if text in YES_VALUES:
        return 1.0
    if text in NO_VALUES:
        return 0.0
    return np.nan

def parse_likert(value):
    if pd.isna(value):
        return np.nan
    text = str(value).strip()
    if not text:
        return np.nan
    match = LIKERT_PATTERN.match(text)
    if match:
        try:
            return float(match.group(1))
        except ValueError:
            return np.nan
    try:
        return float(text)
    except ValueError:
        lowered = text.lower()
        keyword_map = {
            "strongly disagree": 1.0,
            "disagree": 2.0,
            "neither agree nor disagree": 3.0,
            "agree": 4.0,
            "strongly agree": 5.0,
        }
        for keyword, score in keyword_map.items():
            if keyword in lowered:
                return score
    return np.nan

def build_forms_lookup(stage1_forms: pd.DataFrame) -> Dict[str, Dict[str, str]]:
    lookup: Dict[str, Dict[str, str]] = {}
    for row in stage1_forms.itertuples(index=False):
        respondent = str(getattr(row, "respondent", "")).strip()
        if not respondent:
            continue
        short_form = canonicalize_title(getattr(row, "Short Form", ""))
        long_form = canonicalize_title(getattr(row, "Long Form", ""))
        record: Dict[str, str] = {}
        if short_form:
            record["Short"] = short_form
        if long_form:
            record["Long"] = long_form
        if record:
            lookup[respondent] = record
    return lookup

def build_group_title_form_lookup(stimulus_map: pd.DataFrame) -> Dict[tuple[str, str], str]:
    lookup: Dict[tuple[str, str], str] = {}
    for row in stimulus_map.itertuples(index=False):
        group_letter = str(getattr(row, "group", "")).strip()
        if not group_letter:
            continue
        if " " in group_letter:
            group_letter = group_letter.split()[-1]
        group_letter = group_letter.upper()
        title_clean = canonicalize_title(getattr(row, "title", ""))
        form_clean = str(getattr(row, "form", "")).strip().title()
        if group_letter and title_clean:
            lookup[(group_letter, title_clean)] = form_clean
    return lookup

def load_post_survey_raw(post_dir: Path) -> pd.DataFrame:
    frames: list[pd.DataFrame] = []
    for path in sorted(post_dir.rglob("*.csv")):
        if not path.is_file() or path.name.startswith("~$"):
            continue
        df_raw = pd.read_csv(path, dtype=str, encoding="utf-8", on_bad_lines="skip")
        df_norm = merge_duplicate_columns(df_raw)
        participant_col = next((col for col in df_norm.columns if "participant number" in col.lower()), None)
        if participant_col is None:
            raise KeyError(f"Participant number column not found in {path.name}")
        df_norm["respondent"] = df_norm[participant_col].astype(str).str.strip()
        df_norm["respondent"] = df_norm["respondent"].replace({"": np.nan, "nan": np.nan})
        df_norm["respondent"] = df_norm["respondent"].str.replace(r"\.0$", "", regex=True)
        df_norm = df_norm[df_norm["respondent"].notna()]
        if df_norm.empty:
            continue
        timestamp_col = next((col for col in df_norm.columns if col.strip().lower() == "timestamp"), None)
        if timestamp_col:
            timestamp_series = pd.to_datetime(df_norm[timestamp_col], errors="coerce")
            df_norm["submitted_timestamp"] = timestamp_series.dt.tz_localize(None).dt.strftime("%Y-%m-%d %H:%M:%S")
        else:
            df_norm["submitted_timestamp"] = np.nan
        group_match = re.search(r"Group\s+([A-F])", path.stem, re.IGNORECASE)
        df_norm["file_group"] = group_match.group(1).upper() if group_match else np.nan
        try:
            relative_path = path.relative_to(project_root)
            df_norm["source_path"] = relative_path.as_posix()
        except ValueError:
            df_norm["source_path"] = path.as_posix()
        df_norm["questionnaire"] = "Post"
        frames.append(df_norm)
    if not frames:
        return pd.DataFrame()
    combined = pd.concat(frames, ignore_index=True)
    combined["respondent"] = combined["respondent"].astype(str).str.strip()
    combined.drop_duplicates(subset=["respondent", "source_path", "submitted_timestamp"], keep="last", inplace=True)
    return combined

def reshape_post_survey_long(wide_df: pd.DataFrame, value_columns: Iterable[str]) -> pd.DataFrame:
    if wide_df.empty:
        return pd.DataFrame(columns=[
            "respondent", "group", "questionnaire", "submitted_timestamp", "source_path",
            "question_text", "question_code", "response_raw", "file_group", "short_form_title", "long_form_title"
        ])
    records: list[dict] = []
    base_columns = {
        "respondent", "group", "questionnaire", "submitted_timestamp", "source_path", "file_group",
        "short_form_title", "long_form_title"
    }
    for row in wide_df.to_dict("records"):
        base_info = {key: row.get(key) for key in base_columns}
        for column in value_columns:
            value = row.get(column, np.nan)
            records.append({
                **base_info,
                "question_text": column,
                "question_code": extract_question_code(column),
                "response_raw": value,
            })
    return pd.DataFrame(records)

def attach_post_metadata(long_df: pd.DataFrame, metadata: pd.DataFrame, respondent_forms: Dict[str, Dict[str, str]], group_title_form_lookup: Dict[tuple[str, str], str]) -> tuple[pd.DataFrame, pd.DataFrame]:
    if long_df.empty:
        return long_df.copy(), pd.DataFrame()
    meta = metadata.copy()
    meta["question_norm"] = meta["question"].map(normalize_question_text)
    group_columns = [col for col in meta.columns if col.startswith("Group ")]
    merge_columns = ["question_norm", "question", "question_code", "type", "subscale", "category", "accuracy", *group_columns]
    enriched = long_df.copy()
    enriched["question_norm"] = enriched["question_text"].map(normalize_question_text)
    enriched = enriched.merge(meta[merge_columns], on="question_norm", how="left", suffixes=("", "_meta"))
    enriched["question_code"] = enriched["question_code"].fillna(enriched["question_code_meta"])
    issues: list[dict] = []
    stimulus_titles: list[str] = []
    stimulus_forms: list[str] = []
    for _, row in enriched.iterrows():
        respondent_id = str(row.get("respondent", "")).strip()
        group_letter = str(row.get("group", "") or row.get("file_group", "")).strip().upper()
        title_value = ""
        if group_letter:
            group_col = f"Group {group_letter}"
            if group_col in row and pd.notna(row[group_col]):
                title_value = canonicalize_title(row[group_col])
        if not title_value:
            title_value = ""
        form_value = ""
        forms_info = respondent_forms.get(respondent_id, {})
        if title_value:
            for form_label, form_title in forms_info.items():
                if canonicalize_title(form_title) == title_value:
                    form_value = form_label
                    break
            if not form_value and group_letter:
                form_value = group_title_form_lookup.get((group_letter, title_value), "")
        if title_value and not form_value and (str(row.get("type", "")).lower() == "recognition" or str(row.get("subscale", "")).lower() == "recognition"):
            issues.append({
                "respondent": respondent_id,
                "question_code": row.get("question_code"),
                "question_text": row.get("question_text"),
                "issue": "unable_to_resolve_form_for_recognition",
                "group": group_letter,
            })
        stimulus_titles.append(title_value if title_value else np.nan)
        stimulus_forms.append(form_value if form_value else np.nan)
    enriched["stimulus_title"] = stimulus_titles
    enriched["stimulus_form"] = stimulus_forms
    enriched.drop(columns=[col for col in enriched.columns if col.startswith("Group ")], inplace=True)
    enriched.drop(columns=["question_norm", "question_code_meta"], inplace=True, errors="ignore")
    issues_df = pd.DataFrame(issues)
    return enriched, issues_df

def build_recognition_score_frame(stage3_df: pd.DataFrame) -> pd.DataFrame:
    if stage3_df.empty:
        return pd.DataFrame()
    pattern = re.compile(r"^(Long|Short)_(.+)_Post_Recognition_Q(\d+)-(\d+)$")
    frames: list[pd.DataFrame] = []
    for column in stage3_df.columns:
        match = pattern.match(column)
        if not match:
            continue
        form_value, raw_title, question_num, variant = match.groups()
        question_code = f"{int(question_num)}.{variant}"
        method = "stage3_recognition_binary" if variant == "1" else "stage3_recognition_confidence"
        explanation = (
            "Stage 3 recognition correctness (1 = aligns with accuracy expectation, 0 = incorrect)."
            if variant == "1"
            else "Stage 3 recognition confidence parsed to a 0-4 scale."
        )
        frame = stage3_df[["respondent", column]].copy()
        frame.rename(columns={column: "score_value"}, inplace=True)
        frame["respondent"] = frame["respondent"].astype(str).str.strip()
        frame["stimulus_form"] = form_value
        frame["stimulus_title"] = canonicalize_title(raw_title)
        frame["question_code"] = question_code
        frame["score_method"] = method
        frame["score_explanation"] = explanation
        frames.append(frame)
    if not frames:
        return pd.DataFrame()
    recognition_scores = pd.concat(frames, ignore_index=True)
    recognition_scores["score_value"] = pd.to_numeric(recognition_scores["score_value"], errors="coerce")
    mask_confidence = recognition_scores["score_method"] == "stage3_recognition_confidence"
    recognition_scores.loc[mask_confidence, "score_confidence"] = recognition_scores.loc[mask_confidence, "score_value"]
    recognition_scores.loc[~mask_confidence, "score_confidence"] = np.nan
    return recognition_scores

def build_recall_score_frame(results_dir: Path) -> pd.DataFrame:
    frames: list[pd.DataFrame] = []
    file_specs = [
        ("recall_coded_responses_full.csv", "stage5_recall_full"),
        ("recall_coded_responses_key_moment.csv", "stage5_recall_keymoment"),
    ]
    for filename, method in file_specs:
        path = results_dir / filename
        if not path.exists():
            continue
        df = pd.read_csv(path)
        if df.empty:
            continue
        df["respondent"] = df["respondent"].astype(str).str.strip()
        df["question_code"] = df["question_code"].astype(str).str.strip().str.replace(r"^Q", "", regex=True)
        df["stimulus_form"] = df.get("form", "").astype(str).str.title()
        df["stimulus_title"] = df.get("title", "").map(canonicalize_title)
        df["score_value"] = pd.to_numeric(df.get("recall_score"), errors="coerce")
        df["score_confidence"] = pd.to_numeric(df.get("confidence_score"), errors="coerce")
        df["score_explanation"] = df.get("rationale")
        df["score_method"] = method
        frames.append(df[[
            "respondent", "question_code", "stimulus_form", "stimulus_title",
            "score_value", "score_confidence", "score_method", "score_explanation"
        ]])
    if not frames:
        return pd.DataFrame()
    scores = pd.concat(frames, ignore_index=True)
    priority_map = {"stage5_recall_keymoment": 0, "stage5_recall_full": 1}
    scores["priority"] = scores["score_method"].map(priority_map).fillna(2)
    key_cols = ["respondent", "question_code", "stimulus_form", "stimulus_title"]
    scores.sort_values(key_cols + ["priority"], inplace=True)
    scores = scores.drop_duplicates(subset=key_cols, keep="first")
    scores.drop(columns="priority", inplace=True)
    return scores

def append_post_scores(long_df: pd.DataFrame, recognition_scores: pd.DataFrame, recall_scores: pd.DataFrame) -> pd.DataFrame:
    combined = long_df.copy()
    score_frames: list[pd.DataFrame] = []
    for frame in (recognition_scores, recall_scores):
        if frame is not None and not frame.empty:
            temp = frame.copy()
            temp["stimulus_form_key"] = temp["stimulus_form"].fillna("__NA__")
            temp["stimulus_title_key"] = temp["stimulus_title"].fillna("__NA__")
            score_frames.append(temp)
    if not score_frames:
        combined["score_value"] = np.nan
        combined["score_confidence"] = np.nan
        combined["score_method"] = np.nan
        combined["score_explanation"] = np.nan
        return combined
    all_scores = pd.concat(score_frames, ignore_index=True)
    combined["stimulus_form_key"] = combined["stimulus_form"].fillna("__NA__")
    combined["stimulus_title_key"] = combined["stimulus_title"].fillna("__NA__")
    key_cols = ["respondent", "question_code", "stimulus_form_key", "stimulus_title_key"]
    all_scores = all_scores.sort_values(key_cols)
    all_scores = all_scores.drop_duplicates(subset=key_cols, keep="first")
    combined = combined.merge(
        all_scores.drop(columns=["stimulus_form", "stimulus_title"]),
        on=key_cols,
        how="left",
    )
    fallback = all_scores.drop(columns=["stimulus_form_key", "stimulus_title_key"])
    fallback = fallback.drop_duplicates(subset=["respondent", "question_code"], keep="first")
    missing_mask = combined["score_value"].isna() & combined["question_code"].notna()
    if missing_mask.any():
        merged_fallback = (
            combined.loc[missing_mask, ["respondent", "question_code"]]
            .merge(fallback, on=["respondent", "question_code"], how="left")
        )
        combined.loc[missing_mask, ["score_value", "score_confidence", "score_method", "score_explanation"]] = merged_fallback[["score_value", "score_confidence", "score_method", "score_explanation"]].values
    combined.drop(columns=["stimulus_form_key", "stimulus_title_key"], inplace=True)
    return combined

def compute_response_numeric(row) -> float:
    question_type = str(row.get("type", "")).strip().lower()
    if question_type in {"binary", "watch"}:
        return parse_binary(row.get("response_clean"))
    if question_type == "likert":
        return parse_likert(row.get("response_clean"))
    return np.nan

def finalize_columns(df: pd.DataFrame) -> pd.DataFrame:
    df["response_clean"] = df["response_raw"].apply(clean_response)
    df["response_numeric"] = df.apply(compute_response_numeric, axis=1)
    df["submitted_timestamp"] = df["submitted_timestamp"].fillna("")
    ordered_columns = [
        "respondent", "group", "questionnaire", "submitted_timestamp", "source_path",
        "question_code", "question_text", "type", "subscale", "category", "accuracy",
        "stimulus_form", "stimulus_title", "response_raw", "response_clean", "response_numeric",
        "score_value", "score_confidence", "score_method", "score_explanation"
    ]
    for column in ordered_columns:
        if column not in df.columns:
            df[column] = np.nan
    return df.loc[:, ordered_columns]

In [4]:
# Assemble client deliverable dataset
stage1_path = RESULTS_DIR / "uv_stage1.csv"
stage3_path = RESULTS_DIR / "uv_stage3.csv"
metadata_path = DATA_DIR / "post_survey_map.csv"
stimulus_map_path = DATA_DIR / "stimulus_rename.csv"
recall_full_path = RESULTS_DIR / "recall_coded_responses_full.csv"
recall_key_path = RESULTS_DIR / "recall_coded_responses_key_moment.csv"
deliverable_path = RESULTS_DIR / "post_study_survey_responses.csv"

stage1_df = pd.read_csv(stage1_path)
stage3_df = pd.read_csv(stage3_path)
metadata_df = pd.read_csv(metadata_path)
stimulus_map_df = pd.read_csv(stimulus_map_path)

post_raw_df = load_post_survey_raw(POST_DIR)
post_raw_df["respondent"] = post_raw_df["respondent"].astype(str).str.strip()

stage1_subset = stage1_df[["respondent", "group", "Short Form", "Long Form"]].copy()
stage1_subset["respondent"] = stage1_subset["respondent"].astype(str).str.strip()
stage1_subset["group"] = stage1_subset["group"].astype(str).str.strip().str.upper()
stage1_subset["short_form_title"] = stage1_subset["Short Form"].map(canonicalize_title)
stage1_subset["long_form_title"] = stage1_subset["Long Form"].map(canonicalize_title)
stage1_subset.drop(columns=["Short Form", "Long Form"], inplace=True)

post_wide = post_raw_df.merge(stage1_subset, on="respondent", how="left", suffixes=("", "_stage1"))
post_wide["group"] = post_wide["group"].fillna(post_wide.get("file_group"))
post_wide["group"] = post_wide["group"].astype(str).str.strip().str.upper()
post_wide.loc[post_wide["group"].isin({"", "NAN"}), "group"] = np.nan
post_wide["short_form_title"] = post_wide["short_form_title"].fillna("").map(canonicalize_title)
post_wide["long_form_title"] = post_wide["long_form_title"].fillna("").map(canonicalize_title)
post_wide.loc[post_wide["short_form_title"] == "", "short_form_title"] = np.nan
post_wide.loc[post_wide["long_form_title"] == "", "long_form_title"] = np.nan

respondent_forms = build_forms_lookup(stage1_df)
group_title_form_lookup = build_group_title_form_lookup(stimulus_map_df)

base_columns = {
    "respondent",
    "group",
    "questionnaire",
    "submitted_timestamp",
    "source_path",
    "file_group",
    "short_form_title",
    "long_form_title",
}
value_columns = [col for col in post_wide.columns if col not in base_columns and not col.endswith("_stage1") and col not in {"group_stage1"}]
value_columns = list(dict.fromkeys(value_columns))

post_long_df = reshape_post_survey_long(post_wide, value_columns)
post_enriched_df, metadata_issues_df = attach_post_metadata(
    post_long_df, metadata_df, respondent_forms, group_title_form_lookup
)

recognition_scores_df = build_recognition_score_frame(stage3_df)
recall_scores_df = build_recall_score_frame(RESULTS_DIR)

post_scored_df = append_post_scores(post_enriched_df, recognition_scores_df, recall_scores_df)
post_final_df = finalize_columns(post_scored_df)

post_final_df["submitted_dt"] = pd.to_datetime(post_final_df["submitted_timestamp"], errors="coerce")
post_final_df["question_code_fill"] = post_final_df["question_code"].fillna("__NA__")
post_final_df["question_text_fill"] = post_final_df["question_text"].fillna("__NA__")
post_final_df["source_path_fill"] = post_final_df["source_path"].fillna("__NA__")
post_final_df.sort_values(
    ["respondent", "question_code_fill", "question_text_fill", "source_path_fill", "submitted_dt"],
    inplace=True,
)
dedupe_subset = ["respondent", "question_code_fill", "question_text_fill", "source_path_fill"]
before_dedupe_rows = len(post_final_df)
post_final_df = post_final_df.drop_duplicates(subset=dedupe_subset, keep="last")
timestamp_dedupes = before_dedupe_rows - len(post_final_df)

post_final_df.drop(columns=["submitted_dt", "question_code_fill", "question_text_fill", "source_path_fill"], inplace=True)
if timestamp_dedupes:
    print(
        f"Removed {timestamp_dedupes} duplicate rows (kept latest submission per respondent/question).",
    )

print(f"Post deliverable rows: {len(post_final_df):,}")
post_final_df.head()

Removed 73 duplicate rows (kept latest submission per respondent/question).
Post deliverable rows: 6,059


,respondent,group,questionnaire,submitted_timestamp,source_path,question_code,question_text,type,subscale,category,accuracy,stimulus_form,stimulus_title,response_raw,response_clean,response_numeric,score_value,score_confidence,score_method,score_explanation
2557,1,C,Post,2025-10-15 13:56:04,data/Post/Group C_Post Viewing Questionnaire P...,1.1,1.1. Did you see this scene in the videos you ...,binary,recognition,key,hit,Short,The Town,No,No,0.0,0.0,NaN,stage3_recognition_binary,Stage 3 recognition correctness (1 = aligns wi...
2558,1,C,Post,2025-10-15 13:56:04,data/Post/Group C_Post Viewing Questionnaire P...,1.2,1.2. How confident are you in your answer? Ple...,likert,recognition,key,hit,Short,The Town,4 = Extremely confident,4 = Extremely confident,4.0,4.0,4.0,stage3_recognition_confidence,Stage 3 recognition confidence parsed to a 0-4...
2559,1,C,Post,2025-10-15 13:56:04,data/Post/Group C_Post Viewing Questionnaire P...,1.3,1.3. This scene was part of the videos you wat...,multiple,recognition,key,hit,Short,The Town,2 = The scene didn't seem memorable,2 = The scene didn't seem memorable,NaN,NaN,NaN,NaN,NaN
2620,1,C,Post,2025-10-15 13:56:04,data/Post/Group C_Post Viewing Questionnaire P...,10,10. Did you understand the scene while watchin...,NaN,NaN,NaN,NaN,NaN,NaN,3 = Understood it well,3 = Understood it well,NaN,NaN,NaN,NaN,NaN
2584,1,C,Post,2025-10-15 13:56:04,data/Post/Group C_Post Viewing Questionnaire P...,10.1,10.1. Did you see this scene in the videos you...,binary,recognition,distractor,hit,NaN,Ironman,NaN,NaN,NaN,NaN,NaN,stage3_recognition_binary,Stage 3 recognition correctness (1 = aligns wi...


In [5]:
deliverable_path.parent.mkdir(parents=True, exist_ok=True)

# Add an extra step here to drop NA values in response column
post_final_df = post_final_df[post_final_df["response_raw"].notna()]

post_final_df.to_csv(deliverable_path, index=False)

print(f"Saved CSV to {deliverable_path} ({len(post_final_df):,} rows).")

Saved CSV to c:\Users\ashra\Documents\NeuralSense\NeuralData\clients\544_WBD_CXCU\results\post_study_survey_responses.csv (3,910 rows).


In [6]:
validation_summary = {
    "rows": len(post_final_df),
    "unique_respondents": post_final_df["respondent"].nunique(),
    "unique_question_codes": post_final_df["question_code"].nunique(),
    "rows_missing_question_code": int(post_final_df["question_code"].isna().sum()),
    "rows_missing_type": int(post_final_df["type"].isna().sum()),
    "rows_with_scores": int(post_final_df["score_value"].notna().sum()),
    "rows_missing_group": int(post_final_df["group"].isna().sum()),
}
pd.Series(validation_summary)

rows                          3910
unique_respondents              83
unique_question_codes           59
rows_missing_question_code     166
rows_missing_type              162
rows_with_scores              1584
rows_missing_group               0
dtype: int64

In [7]:
duplicate_count = post_final_df.assign(
    question_code_fill=post_final_df["question_code"].fillna("__NA__"),
    question_text_fill=post_final_df["question_text"].fillna("__NA__"),
).duplicated(subset=["respondent", "question_code_fill", "question_text_fill", "source_path"]).sum()
duplicate_count

np.int64(0)

In [8]:
duplicates_df = post_final_df.assign(
    question_code_fill=post_final_df["question_code"].fillna("__NA__"),
    question_text_fill=post_final_df["question_text"].fillna("__NA__"),
)
duplicates_df[duplicates_df.duplicated(subset=["respondent", "question_code_fill", "question_text_fill", "source_path"], keep=False)][
    ["respondent", "question_code", "question_text", "source_path", "response_raw"]
].head(10)

,respondent,question_code,question_text,source_path,response_raw


In [9]:
duplicate_keys_mask = duplicates_df.duplicated(subset=["respondent", "question_code_fill", "question_text_fill", "source_path"], keep=False)
duplicate_group_sizes = (
    duplicates_df.loc[duplicate_keys_mask]
    .groupby(["respondent", "question_code_fill", "question_text_fill", "source_path"])
)
duplicate_group_sizes.size().head()

Series([], dtype: int64)

In [11]:
list(duplicates_df.columns)

['respondent',
 'group',
 'questionnaire',
 'submitted_timestamp',
 'source_path',
 'question_code',
 'question_text',
 'type',
 'subscale',
 'category',
 'accuracy',
 'stimulus_form',
 'stimulus_title',
 'response_raw',
 'response_clean',
 'response_numeric',
 'score_value',
 'score_confidence',
 'score_method',
 'score_explanation',
 'question_code_fill',
 'question_text_fill']

In [ ]:
duplicate_keys = duplicates_df.duplicated(subset=["respondent", "question_code_fill", "question_text_fill", "source_path"], keep=False)
duplicate_groups = duplicates_df.loc[duplicate_keys].groupby([
    "respondent",
    "question_code_fill",
    "question_text_fill",
    "source_path",
])
first_key = next(iter(duplicate_groups.groups))
example_rows = duplicates_df.loc[duplicate_groups.groups[first_key]]
example_rows

,respondent,group,questionnaire,submitted_timestamp,source_path,question_code,question_text,type,subscale,category,...,stimulus_title,response_raw,response_clean,response_numeric,score_value,score_confidence,score_method,score_explanation,question_code_fill,question_text_fill
4528,100,E,Post,2025-10-23 22:44:28,data/Post/Group E_ Post Viewing Questionnaire ...,1.1,1.1. Did you see this scene in the videos you ...,binary,recognition,key,...,Mad Max,Yes,Yes,1.0,1.0,NaN,stage3_recognition_binary,Stage 3 recognition correctness (1 = aligns wi...,1.1,1.1. Did you see this scene in the videos you ...
5039,100,E,Post,2025-10-25 16:21:27,data/Post/Group E_ Post Viewing Questionnaire ...,1.1,1.1. Did you see this scene in the videos you ...,binary,recognition,key,...,Mad Max,Yes,Yes,1.0,1.0,NaN,stage3_recognition_binary,Stage 3 recognition correctness (1 = aligns wi...,1.1,1.1. Did you see this scene in the videos you ...


In [ ]:
display_columns = [
    "respondent",
    "group",
    "questionnaire",
    "submitted_timestamp",
    "question_code",
    "question_text",
    "source_path",
    "response_raw",
    "response_clean",
    "response_numeric",
    "type",
    "category",
    "subscale",
    "accuracy",
    "stimulus_form",
    "stimulus_title",
    "score_value",
    "score_confidence",
    "score_method",
    "score_explanation",
]
comparison_columns = [
    column
    for column in duplicates_df.columns
    if column not in {
        "question_code_fill",
        "question_text_fill",
    }
]
duplicate_keys_mask = duplicates_df.duplicated(subset=["respondent", "question_code_fill", "question_text_fill", "source_path"], keep=False)
duplicate_rows = duplicates_df.loc[duplicate_keys_mask]
print(f"Duplicate rows captured: {len(duplicate_rows):,}")
grouped_duplicates = duplicate_rows.groupby([
    "respondent",
    "question_code_fill",
    "question_text_fill",
    "source_path",
])
example_tables = []
diagnostic_examples = []
for key, group in grouped_duplicates:
    subset = group.loc[:, comparison_columns]
    differing_columns = [
        column
        for column in comparison_columns
        if subset[column].nunique(dropna=False) > 1
]
    if not differing_columns:
        diagnostic_examples.append(group)
        continue
    example_frame = group.loc[:, display_columns].copy()
    duplicate_label = " | ".join(str(value) for value in key)
    example_frame.insert(0, "different_columns", ", ".join(sorted(differing_columns)))
    example_frame.insert(0, "duplicate_key", duplicate_label)
    example_frame.insert(0, "example", f"Example {len(example_tables) + 1}")
    example_tables.append(example_frame)
    if len(example_tables) == 3:
        break
if example_tables:
    examples_df = pd.concat(example_tables, ignore_index=True)
    examples_df
else:
    print("No duplicate groups with differing values found.")
    print(f"Groups inspected: {len(diagnostic_examples)}")

Duplicate rows captured: 0
No duplicate groups with differing values found.
Groups inspected: 0


In [ ]:
issue_summary = metadata_issues_df.groupby(['issue', 'question_code']).size().sort_values(ascending=False)
issue_summary.head(10)

issue                                   question_code
unable_to_resolve_form_for_recognition  11.2             84
                                        11.1             84
                                        6.3              84
                                        8.1              84
                                        11.3             84
                                        2.1              84
                                        6.2              84
                                        6.1              84
                                        2.3              84
                                        2.2              84
dtype: int64

In [ ]:
metadata_issues_df.groupby(['question_code', 'group']).size().unstack(fill_value=0)

group,A,B,C,D,E,F
question_code,,,,,,
10.1,0,10,15,0,0,0
10.2,0,10,15,0,0,0
10.3,0,10,15,0,0,0
11.1,12,10,15,18,15,14
11.2,12,10,15,18,15,14
11.3,12,10,15,18,15,14
12.1,0,10,15,0,0,0
12.2,0,10,15,0,0,0
12.3,0,10,15,0,0,0


In [ ]:
metadata_issues_df.head()

,respondent,question_code,question_text,issue,group
0,3,2.1,2.1. Did you see this scene in the videos you ...,unable_to_resolve_form_for_recognition,A
1,3,2.2,2.2. How confident are you in your answer? Ple...,unable_to_resolve_form_for_recognition,A
2,3,2.3,2.3. This scene was not in the videos you watc...,unable_to_resolve_form_for_recognition,A
3,3,6.1,6.1. Did you see this scene in the videos you ...,unable_to_resolve_form_for_recognition,A
4,3,6.2,6.2. How confident are you in your answer? Ple...,unable_to_resolve_form_for_recognition,A


In [ ]:
metadata_issues_df['group'].value_counts()

group
C    270
D    216
B    180
E    180
F    168
A    144
Name: count, dtype: int64

In [ ]:
stage1_subset.loc[stage1_subset['respondent'] == '3']

,respondent,group,short_form_title,long_form_title
4,3,A,Mad Max,The Town


In [ ]:
forms_info = respondent_forms.get('3')
forms_info